In [1]:
import requests
import xml.etree.ElementTree as ET
from collections import defaultdict
import time

In [2]:
def get_token():
    #get access token
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    data = """<?xml version="1.0" encoding="utf-8"?>
    <soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
      <soap12:Body>
        <Open xmlns="http://bvdep.com/webservices/">
          <username>iGenieWS123</username>
          <password>jnecyu4wa!</password>
        </Open>
      </soap12:Body>
    </soap12:Envelope>"""

    response = requests.post(url, headers=headers, data=data)

    if response.status_code != requests.codes.ok:
        print(response.text)
        return None

    OpenResponse = ET.fromstring(response.text)
    token = OpenResponse[0][0][0].text
    print(token)
    return token

In [3]:
def close_connection(token):
    #get access token
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    data = """<?xml version="1.0" encoding="utf-8"?>
<soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
  <soap12:Body>
    <Close xmlns="http://bvdep.com/webservices/">
      <sessionHandle>{}</sessionHandle>
    </Close>
  </soap12:Body>
</soap12:Envelope>""".format(token)

    response = requests.post(url, headers=headers, data=data)

In [4]:
def find_by_bvd_id(token, bvdid):
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    #FindByBvdId
    data = """<?xml version="1.0" encoding="utf-8"?>
    <soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
      <soap12:Body>
        <FindByBVDId xmlns="http://bvdep.com/webservices/">
          <sessionHandle>{}</sessionHandle>
          <id>{}</id>
        </FindByBVDId>
      </soap12:Body>
    </soap12:Envelope>""".format(token,bvdid)

    response = requests.post(url, headers=headers, data=data)

    if response.status_code != requests.codes.ok:
        print(response.text)
        return None

    FindByBvdId = ET.fromstring(response.text)
    SelectionResult_token = FindByBvdId[0][0][0][0].text
    print(SelectionResult_token)
    return SelectionResult_token

In [5]:
def get_data(token, SelectionResult_token, query, field, constituent):        
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    data = """<?xml version="1.0" encoding="utf-8"?>
<soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
  <soap12:Body>
    <GetData xmlns="http://bvdep.com/webservices/">
      <sessionHandle>{0}</sessionHandle>
      <selection>
        <Token>{1}</Token>
        <SelectionCount>1</SelectionCount>
      </selection>
      <query>{2}</query>
      <fromRecord>0</fromRecord>
      <nrRecords>5</nrRecords>
      <resultFormat>CSV</resultFormat>
    </GetData>
  </soap12:Body>
</soap12:Envelope>""".format(token,SelectionResult_token,query)

    response = requests.post(url, headers=headers, data=data)

    if response.status_code != requests.codes.ok:
        print(response.text)
        with open("./results/failed.txt", 'a') as f:
            f.write("{} \n".format(field))
        return None
    
    with open("./results/success.txt", 'a') as f:
            f.write("{} \n".format(field))

    result = response.text
    file_name = "./results/{}_{}.xml".format(constituent,field)
    with open(file_name, 'w') as f:
        f.write(result)
    return True

In [7]:
def get_available_fields():
    all_fields = defaultdict(list)

    available_fields = ET.parse(r'available_fields.xml')
    fields = available_fields.getroot()

    #i = 0
    for child in fields:
        #if i == 20:
            #break
        if len(child) > 0:
            document = {"code":child.attrib["code"], "dimension":child[0].attrib["default"]}
        else:
            document = {"code":child.attrib["code"]}

        all_fields[child.attrib["code"].split("_")[0]].append(document)
        #i += 1  
    return all_fields

In [7]:
def main():
    errors = 0
    all_fields = get_available_fields()
    token = get_token()
    print(token)
    selection_token = find_by_bvd_id(token)
    
    for key in all_fields.keys():
        if errors == 2:
            print("2 errors")
            return
        
        query = "SELECT "
        middle = []
        for item in all_fields[key]:
            part = item["code"] + " "
            if "dimension" in item:
                part += "USING [Parameters.RepeatingDimension={};]".format(item["dimension"])
            middle.append(part)
        query += ", ".join(middle) + " FROM RemoteAccess.A"

        get_data_result = get_data(token,selection_token,query,key,"BMW")
        
        if not get_data_result:
            errors += 1
            close_connection(token)
            time.sleep(900)
            token = get_token()
            if not token:
                print("Couldn't get second token")
                return
            print(token)
            selection_token = find_by_bvd_id(token)
            

        time.sleep(60)

In [ ]:
main()

In [6]:
def get_custom_data(query, bvdid, data, constituent):
    token = get_token()
    print(token)
    selection_token = find_by_bvd_id(token, bvdid)
    
    get_data_result = get_data(token,selection_token,query,data,constituent)
    close_connection(token)

In [12]:
q = """DEFINE P1 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=0;],P2 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=0;],P3 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=1;],P4 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=1;],P5 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=2;],P6 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=2;],P7 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=3;],P8 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=3;],P9 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=4;],P10 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=4;],P11 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=5;],P12 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=5;],P13 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=6;],P14 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=6;],P15 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=7;],P16 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=7;],P17 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=8;],P18 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=8;],P19 AS [Parameters.Currency=EUR;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=9;],P20 AS [Parameters.Currency=USD;Parameters.Units=3;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=9;],P21 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=0;],P22 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=1;],P23 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=2;],P24 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=3;],P25 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=4;],P26 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=5;],P27 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=6;],P28 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=7;],P29 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=8;],P30 AS [Parameters.Currency=SESSION;Parameters.Units=0;Parameters.DimensionSelName=NrOfYears;NrOfYears.Index=9;]; SELECT LINE GENERAL_INFO.NAME AS NAME, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P1 AS OPRE, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P2 AS OPRE_1, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P3 AS OPRE_2, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P4 AS OPRE_3, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P5 AS OPRE_4, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P6 AS OPRE_5, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P7 AS OPRE_6, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P8 AS OPRE_7, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P9 AS OPRE_8, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P10 AS OPRE_9, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P11 AS OPRE_10, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P12 AS OPRE_11, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P13 AS OPRE_12, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P14 AS OPRE_13, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P15 AS OPRE_14, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P16 AS OPRE_15, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P17 AS OPRE_16, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P18 AS OPRE_17, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P19 AS OPRE_18, MODEL [Unified]LINE REPORT_PROFILE.OPREV USING P20 AS OPRE_19, MODEL [Unified]LINE REPORT_PROFILE.429 USING P1 AS PLBT, MODEL [Unified]LINE REPORT_PROFILE.429 USING P2 AS PLBT_1, MODEL [Unified]LINE REPORT_PROFILE.429 USING P3 AS PLBT_2, MODEL [Unified]LINE REPORT_PROFILE.429 USING P4 AS PLBT_3, MODEL [Unified]LINE REPORT_PROFILE.429 USING P5 AS PLBT_4, MODEL [Unified]LINE REPORT_PROFILE.429 USING P6 AS PLBT_5, MODEL [Unified]LINE REPORT_PROFILE.429 USING P7 AS PLBT_6, MODEL [Unified]LINE REPORT_PROFILE.429 USING P8 AS PLBT_7, MODEL [Unified]LINE REPORT_PROFILE.429 USING P9 AS PLBT_8, MODEL [Unified]LINE REPORT_PROFILE.429 USING P10 AS PLBT_9, MODEL [Unified]LINE REPORT_PROFILE.429 USING P11 AS PLBT_10, MODEL [Unified]LINE REPORT_PROFILE.429 USING P12 AS PLBT_11, MODEL [Unified]LINE REPORT_PROFILE.429 USING P13 AS PLBT_12, MODEL [Unified]LINE REPORT_PROFILE.429 USING P14 AS PLBT_13, MODEL [Unified]LINE REPORT_PROFILE.429 USING P15 AS PLBT_14, MODEL [Unified]LINE REPORT_PROFILE.429 USING P16 AS PLBT_15, MODEL [Unified]LINE REPORT_PROFILE.429 USING P17 AS PLBT_16, MODEL [Unified]LINE REPORT_PROFILE.429 USING P18 AS PLBT_17, MODEL [Unified]LINE REPORT_PROFILE.429 USING P19 AS PLBT_18, MODEL [Unified]LINE REPORT_PROFILE.429 USING P20 AS PLBT_19, MODEL [Unified]LINE REPORT_PROFILE.433 USING P1 AS PL, MODEL [Unified]LINE REPORT_PROFILE.433 USING P2 AS PL_1, MODEL [Unified]LINE REPORT_PROFILE.433 USING P3 AS PL_2, MODEL [Unified]LINE REPORT_PROFILE.433 USING P4 AS PL_3, MODEL [Unified]LINE REPORT_PROFILE.433 USING P5 AS PL_4, MODEL [Unified]LINE REPORT_PROFILE.433 USING P6 AS PL_5, MODEL [Unified]LINE REPORT_PROFILE.433 USING P7 AS PL_6, MODEL [Unified]LINE REPORT_PROFILE.433 USING P8 AS PL_7, MODEL [Unified]LINE REPORT_PROFILE.433 USING P9 AS PL_8, MODEL [Unified]LINE REPORT_PROFILE.433 USING P10 AS PL_9, MODEL [Unified]LINE REPORT_PROFILE.433 USING P11 AS PL_10, MODEL [Unified]LINE REPORT_PROFILE.433 USING P12 AS PL_11, MODEL [Unified]LINE REPORT_PROFILE.433 USING P13 AS PL_12, MODEL [Unified]LINE REPORT_PROFILE.433 USING P14 AS PL_13, MODEL [Unified]LINE REPORT_PROFILE.433 USING P15 AS PL_14, MODEL [Unified]LINE REPORT_PROFILE.433 USING P16 AS PL_15, MODEL [Unified]LINE REPORT_PROFILE.433 USING P17 AS PL_16, MODEL [Unified]LINE REPORT_PROFILE.433 USING P18 AS PL_17, MODEL [Unified]LINE REPORT_PROFILE.433 USING P19 AS PL_18, MODEL [Unified]LINE REPORT_PROFILE.433 USING P20 AS PL_19, MODEL [Unified]LINE REPORT_PROFILE.438 USING P1 AS CF, MODEL [Unified]LINE REPORT_PROFILE.438 USING P2 AS CF_1, MODEL [Unified]LINE REPORT_PROFILE.438 USING P3 AS CF_2, MODEL [Unified]LINE REPORT_PROFILE.438 USING P4 AS CF_3, MODEL [Unified]LINE REPORT_PROFILE.438 USING P5 AS CF_4, MODEL [Unified]LINE REPORT_PROFILE.438 USING P6 AS CF_5, MODEL [Unified]LINE REPORT_PROFILE.438 USING P7 AS CF_6, MODEL [Unified]LINE REPORT_PROFILE.438 USING P8 AS CF_7, MODEL [Unified]LINE REPORT_PROFILE.438 USING P9 AS CF_8, MODEL [Unified]LINE REPORT_PROFILE.438 USING P10 AS CF_9, MODEL [Unified]LINE REPORT_PROFILE.438 USING P11 AS CF_10, MODEL [Unified]LINE REPORT_PROFILE.438 USING P12 AS CF_11, MODEL [Unified]LINE REPORT_PROFILE.438 USING P13 AS CF_12, MODEL [Unified]LINE REPORT_PROFILE.438 USING P14 AS CF_13, MODEL [Unified]LINE REPORT_PROFILE.438 USING P15 AS CF_14, MODEL [Unified]LINE REPORT_PROFILE.438 USING P16 AS CF_15, MODEL [Unified]LINE REPORT_PROFILE.438 USING P17 AS CF_16, MODEL [Unified]LINE REPORT_PROFILE.438 USING P18 AS CF_17, MODEL [Unified]LINE REPORT_PROFILE.438 USING P19 AS CF_18, MODEL [Unified]LINE REPORT_PROFILE.438 USING P20 AS CF_19, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P1 AS TOAS, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P2 AS TOAS_1, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P3 AS TOAS_2, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P4 AS TOAS_3, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P5 AS TOAS_4, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P6 AS TOAS_5, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P7 AS TOAS_6, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P8 AS TOAS_7, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P9 AS TOAS_8, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P10 AS TOAS_9, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P11 AS TOAS_10, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P12 AS TOAS_11, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P13 AS TOAS_12, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P14 AS TOAS_13, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P15 AS TOAS_14, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P16 AS TOAS_15, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P17 AS TOAS_16, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P18 AS TOAS_17, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P19 AS TOAS_18, MODEL [Unified]LINE REPORT_PROFILE.TOTASS USING P20 AS TOAS_19, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P1 AS SHFD, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P2 AS SHFD_1, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P3 AS SHFD_2, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P4 AS SHFD_3, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P5 AS SHFD_4, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P6 AS SHFD_5, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P7 AS SHFD_6, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P8 AS SHFD_7, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P9 AS SHFD_8, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P10 AS SHFD_9, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P11 AS SHFD_10, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P12 AS SHFD_11, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P13 AS SHFD_12, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P14 AS SHFD_13, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P15 AS SHFD_14, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P16 AS SHFD_15, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P17 AS SHFD_16, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P18 AS SHFD_17, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P19 AS SHFD_18, MODEL [Unified]LINE REPORT_PROFILE.SHAFUNDS USING P20 AS SHFD_19, MODEL [Unified]LINE REPORT_PROFILE.500 USING P21 AS CURR, MODEL [Unified]LINE REPORT_PROFILE.500 USING P22 AS CURR_1, MODEL [Unified]LINE REPORT_PROFILE.500 USING P23 AS CURR_2, MODEL [Unified]LINE REPORT_PROFILE.500 USING P24 AS CURR_3, MODEL [Unified]LINE REPORT_PROFILE.500 USING P25 AS CURR_4, MODEL [Unified]LINE REPORT_PROFILE.500 USING P26 AS CURR_5, MODEL [Unified]LINE REPORT_PROFILE.500 USING P27 AS CURR_6, MODEL [Unified]LINE REPORT_PROFILE.500 USING P28 AS CURR_7, MODEL [Unified]LINE REPORT_PROFILE.500 USING P29 AS CURR_8, MODEL [Unified]LINE REPORT_PROFILE.500 USING P30 AS CURR_9, MODEL [Unified]LINE REPORT_PROFILE.508 USING P21 AS PRMA, MODEL [Unified]LINE REPORT_PROFILE.508 USING P22 AS PRMA_1, MODEL [Unified]LINE REPORT_PROFILE.508 USING P23 AS PRMA_2, MODEL [Unified]LINE REPORT_PROFILE.508 USING P24 AS PRMA_3, MODEL [Unified]LINE REPORT_PROFILE.508 USING P25 AS PRMA_4, MODEL [Unified]LINE REPORT_PROFILE.508 USING P26 AS PRMA_5, MODEL [Unified]LINE REPORT_PROFILE.508 USING P27 AS PRMA_6, MODEL [Unified]LINE REPORT_PROFILE.508 USING P28 AS PRMA_7, MODEL [Unified]LINE REPORT_PROFILE.508 USING P29 AS PRMA_8, MODEL [Unified]LINE REPORT_PROFILE.508 USING P30 AS PRMA_9, MODEL [Unified]LINE REPORT_PROFILE.509 USING P21 AS RSHF, MODEL [Unified]LINE REPORT_PROFILE.509 USING P22 AS RSHF_1, MODEL [Unified]LINE REPORT_PROFILE.509 USING P23 AS RSHF_2, MODEL [Unified]LINE REPORT_PROFILE.509 USING P24 AS RSHF_3, MODEL [Unified]LINE REPORT_PROFILE.509 USING P25 AS RSHF_4, MODEL [Unified]LINE REPORT_PROFILE.509 USING P26 AS RSHF_5, MODEL [Unified]LINE REPORT_PROFILE.509 USING P27 AS RSHF_6, MODEL [Unified]LINE REPORT_PROFILE.509 USING P28 AS RSHF_7, MODEL [Unified]LINE REPORT_PROFILE.509 USING P29 AS RSHF_8, MODEL [Unified]LINE REPORT_PROFILE.509 USING P30 AS RSHF_9, MODEL [Unified]LINE REPORT_PROFILE.510 USING P21 AS RCEM, MODEL [Unified]LINE REPORT_PROFILE.510 USING P22 AS RCEM_1, MODEL [Unified]LINE REPORT_PROFILE.510 USING P23 AS RCEM_2, MODEL [Unified]LINE REPORT_PROFILE.510 USING P24 AS RCEM_3, MODEL [Unified]LINE REPORT_PROFILE.510 USING P25 AS RCEM_4, MODEL [Unified]LINE REPORT_PROFILE.510 USING P26 AS RCEM_5, MODEL [Unified]LINE REPORT_PROFILE.510 USING P27 AS RCEM_6, MODEL [Unified]LINE REPORT_PROFILE.510 USING P28 AS RCEM_7, MODEL [Unified]LINE REPORT_PROFILE.510 USING P29 AS RCEM_8, MODEL [Unified]LINE REPORT_PROFILE.510 USING P30 AS RCEM_9, MODEL [Unified]LINE REPORT_PROFILE.503 USING P21 AS SOLR, MODEL [Unified]LINE REPORT_PROFILE.503 USING P22 AS SOLR_1, MODEL [Unified]LINE REPORT_PROFILE.503 USING P23 AS SOLR_2, MODEL [Unified]LINE REPORT_PROFILE.503 USING P24 AS SOLR_3, MODEL [Unified]LINE REPORT_PROFILE.503 USING P25 AS SOLR_4, MODEL [Unified]LINE REPORT_PROFILE.503 USING P26 AS SOLR_5, MODEL [Unified]LINE REPORT_PROFILE.503 USING P27 AS SOLR_6, MODEL [Unified]LINE REPORT_PROFILE.503 USING P28 AS SOLR_7, MODEL [Unified]LINE REPORT_PROFILE.503 USING P29 AS SOLR_8, MODEL [Unified]LINE REPORT_PROFILE.503 USING P30 AS SOLR_9, MODEL [Unified]LINE REPORT_PROFILE.611 USING P21 AS 611, MODEL [Unified]LINE REPORT_PROFILE.611 USING P22 AS 611_1, MODEL [Unified]LINE REPORT_PROFILE.611 USING P23 AS 611_2, MODEL [Unified]LINE REPORT_PROFILE.611 USING P24 AS 611_3, MODEL [Unified]LINE REPORT_PROFILE.611 USING P25 AS 611_4, MODEL [Unified]LINE REPORT_PROFILE.611 USING P26 AS 611_5, MODEL [Unified]LINE REPORT_PROFILE.611 USING P27 AS 611_6, MODEL [Unified]LINE REPORT_PROFILE.611 USING P28 AS 611_7, MODEL [Unified]LINE REPORT_PROFILE.611 USING P29 AS 611_8, MODEL [Unified]LINE REPORT_PROFILE.611 USING P30 AS 611_9, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P21 AS EMPL, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P22 AS EMPL_1, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P23 AS EMPL_2, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P24 AS EMPL_3, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P25 AS EMPL_4, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P26 AS EMPL_5, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P27 AS EMPL_6, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P28 AS EMPL_7, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P29 AS EMPL_8, MODEL [Unified]LINE REPORT_PROFILE.EMPL USING P30 AS EMPL_9, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P2 AS 191200, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P4 AS 191200_1, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P6 AS 191200_2, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P8 AS 191200_3, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P10 AS 191200_4, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P12 AS 191200_5, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P14 AS 191200_6, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P16 AS 191200_7, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P18 AS 191200_8, MODEL [Unified]LINE REPORT_PROFILE.142030 USING P20 AS 191200_9, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P21 AS 195090, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P22 AS 195090_1, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P23 AS 195090_2, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P24 AS 195090_3, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P25 AS 195090_4, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P26 AS 195090_5, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P27 AS 195090_6, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P28 AS 195090_7, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P29 AS 195090_8, MODEL [Unified]LINE REPORT_PROFILE.144008 USING P30 AS 195090_9, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P21 AS 195190, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P22 AS 195190_1, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P23 AS 195190_2, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P24 AS 195190_3, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P25 AS 195190_4, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P26 AS 195190_5, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P27 AS 195190_6, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P28 AS 195190_7, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P29 AS 195190_8, MODEL [Unified]LINE REPORT_PROFILE.144018 USING P30 AS 195190_9, MODEL [Unified]LINE REPORT_PROFILE.511 USING P21 AS RTAS, MODEL [Unified]LINE REPORT_PROFILE.511 USING P22 AS RTAS_1, MODEL [Unified]LINE REPORT_PROFILE.511 USING P23 AS RTAS_2, MODEL [Unified]LINE REPORT_PROFILE.511 USING P24 AS RTAS_3, MODEL [Unified]LINE REPORT_PROFILE.511 USING P25 AS RTAS_4, MODEL [Unified]LINE REPORT_PROFILE.511 USING P26 AS RTAS_5, MODEL [Unified]LINE REPORT_PROFILE.511 USING P27 AS RTAS_6, MODEL [Unified]LINE REPORT_PROFILE.511 USING P28 AS RTAS_7, MODEL [Unified]LINE REPORT_PROFILE.511 USING P29 AS RTAS_8, MODEL [Unified]LINE REPORT_PROFILE.511 USING P30 AS RTAS_9, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P21 AS 195300, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P22 AS 195300_1, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P23 AS 195300_2, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P24 AS 195300_3, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P25 AS 195300_4, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P26 AS 195300_5, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P27 AS 195300_6, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P28 AS 195300_7, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P29 AS 195300_8, MODEL [Unified]LINE REPORT_PROFILE.144029 USING P30 AS 195300_9, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P2 AS 208777, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P4 AS 208777_1, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P6 AS 208777_2, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P8 AS 208777_3, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P10 AS 208777_4, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P12 AS 208777_5, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P14 AS 208777_6, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P16 AS 208777_7, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P18 AS 208777_8, MODEL [Unified]LINE REPORT_PROFILE.208777 USING P20 AS 208777_9, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P21 AS 290015, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P22 AS 290015_1, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P23 AS 290015_2, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P24 AS 290015_3, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P25 AS 290015_4, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P26 AS 290015_5, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P27 AS 290015_6, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P28 AS 290015_7, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P29 AS 290015_8, MODEL [Unified]LINE REPORT_PROFILE.290015 USING P30 AS 290015_9, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P21 AS 291012, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P22 AS 291012_1, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P23 AS 291012_2, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P24 AS 291012_3, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P25 AS 291012_4, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P26 AS 291012_5, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P27 AS 291012_6, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P28 AS 291012_7, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P29 AS 291012_8, MODEL [Unified]LINE REPORT_PROFILE.291012 USING P30 AS 291012_9, MODEL [C]LINE GLOBAL.404 USING P2 AS FIAS, MODEL [C]LINE GLOBAL.404 USING P4 AS FIAS_1, MODEL [C]LINE GLOBAL.404 USING P6 AS FIAS_2, MODEL [C]LINE GLOBAL.404 USING P8 AS FIAS_3, MODEL [C]LINE GLOBAL.404 USING P10 AS FIAS_4, MODEL [C]LINE GLOBAL.404 USING P12 AS FIAS_5, MODEL [C]LINE GLOBAL.404 USING P14 AS FIAS_6, MODEL [C]LINE GLOBAL.404 USING P16 AS FIAS_7, MODEL [C]LINE GLOBAL.404 USING P18 AS FIAS_8, MODEL [C]LINE GLOBAL.404 USING P20 AS FIAS_9, MODEL [C]LINE GLOBAL.405 USING P2 AS IFAS, MODEL [C]LINE GLOBAL.405 USING P4 AS IFAS_1, MODEL [C]LINE GLOBAL.405 USING P6 AS IFAS_2, MODEL [C]LINE GLOBAL.405 USING P8 AS IFAS_3, MODEL [C]LINE GLOBAL.405 USING P10 AS IFAS_4, MODEL [C]LINE GLOBAL.405 USING P12 AS IFAS_5, MODEL [C]LINE GLOBAL.405 USING P14 AS IFAS_6, MODEL [C]LINE GLOBAL.405 USING P16 AS IFAS_7, MODEL [C]LINE GLOBAL.405 USING P18 AS IFAS_8, MODEL [C]LINE GLOBAL.405 USING P20 AS IFAS_9, MODEL [C]LINE GLOBAL.406 USING P2 AS TFAS, MODEL [C]LINE GLOBAL.406 USING P4 AS TFAS_1, MODEL [C]LINE GLOBAL.406 USING P6 AS TFAS_2, MODEL [C]LINE GLOBAL.406 USING P8 AS TFAS_3, MODEL [C]LINE GLOBAL.406 USING P10 AS TFAS_4, MODEL [C]LINE GLOBAL.406 USING P12 AS TFAS_5, MODEL [C]LINE GLOBAL.406 USING P14 AS TFAS_6, MODEL [C]LINE GLOBAL.406 USING P16 AS TFAS_7, MODEL [C]LINE GLOBAL.406 USING P18 AS TFAS_8, MODEL [C]LINE GLOBAL.406 USING P20 AS TFAS_9, MODEL [C]LINE GLOBAL.407 USING P2 AS OFAS, MODEL [C]LINE GLOBAL.407 USING P4 AS OFAS_1, MODEL [C]LINE GLOBAL.407 USING P6 AS OFAS_2, MODEL [C]LINE GLOBAL.407 USING P8 AS OFAS_3, MODEL [C]LINE GLOBAL.407 USING P10 AS OFAS_4, MODEL [C]LINE GLOBAL.407 USING P12 AS OFAS_5, MODEL [C]LINE GLOBAL.407 USING P14 AS OFAS_6, MODEL [C]LINE GLOBAL.407 USING P16 AS OFAS_7, MODEL [C]LINE GLOBAL.407 USING P18 AS OFAS_8, MODEL [C]LINE GLOBAL.407 USING P20 AS OFAS_9, MODEL [C]LINE GLOBAL.408 USING P2 AS CUAS, MODEL [C]LINE GLOBAL.408 USING P4 AS CUAS_1, MODEL [C]LINE GLOBAL.408 USING P6 AS CUAS_2, MODEL [C]LINE GLOBAL.408 USING P8 AS CUAS_3, MODEL [C]LINE GLOBAL.408 USING P10 AS CUAS_4, MODEL [C]LINE GLOBAL.408 USING P12 AS CUAS_5, MODEL [C]LINE GLOBAL.408 USING P14 AS CUAS_6, MODEL [C]LINE GLOBAL.408 USING P16 AS CUAS_7, MODEL [C]LINE GLOBAL.408 USING P18 AS CUAS_8, MODEL [C]LINE GLOBAL.408 USING P20 AS CUAS_9, MODEL [C]LINE GLOBAL.409 USING P2 AS STOK, MODEL [C]LINE GLOBAL.409 USING P4 AS STOK_1, MODEL [C]LINE GLOBAL.409 USING P6 AS STOK_2, MODEL [C]LINE GLOBAL.409 USING P8 AS STOK_3, MODEL [C]LINE GLOBAL.409 USING P10 AS STOK_4, MODEL [C]LINE GLOBAL.409 USING P12 AS STOK_5, MODEL [C]LINE GLOBAL.409 USING P14 AS STOK_6, MODEL [C]LINE GLOBAL.409 USING P16 AS STOK_7, MODEL [C]LINE GLOBAL.409 USING P18 AS STOK_8, MODEL [C]LINE GLOBAL.409 USING P20 AS STOK_9, MODEL [C]LINE GLOBAL.410 USING P2 AS DEBT, MODEL [C]LINE GLOBAL.410 USING P4 AS DEBT_1, MODEL [C]LINE GLOBAL.410 USING P6 AS DEBT_2, MODEL [C]LINE GLOBAL.410 USING P8 AS DEBT_3, MODEL [C]LINE GLOBAL.410 USING P10 AS DEBT_4, MODEL [C]LINE GLOBAL.410 USING P12 AS DEBT_5, MODEL [C]LINE GLOBAL.410 USING P14 AS DEBT_6, MODEL [C]LINE GLOBAL.410 USING P16 AS DEBT_7, MODEL [C]LINE GLOBAL.410 USING P18 AS DEBT_8, MODEL [C]LINE GLOBAL.410 USING P20 AS DEBT_9, MODEL [C]LINE GLOBAL.411 USING P2 AS OCAS, MODEL [C]LINE GLOBAL.411 USING P4 AS OCAS_1, MODEL [C]LINE GLOBAL.411 USING P6 AS OCAS_2, MODEL [C]LINE GLOBAL.411 USING P8 AS OCAS_3, MODEL [C]LINE GLOBAL.411 USING P10 AS OCAS_4, MODEL [C]LINE GLOBAL.411 USING P12 AS OCAS_5, MODEL [C]LINE GLOBAL.411 USING P14 AS OCAS_6, MODEL [C]LINE GLOBAL.411 USING P16 AS OCAS_7, MODEL [C]LINE GLOBAL.411 USING P18 AS OCAS_8, MODEL [C]LINE GLOBAL.411 USING P20 AS OCAS_9, MODEL [C]LINE GLOBAL.440 USING P2 AS CASH, MODEL [C]LINE GLOBAL.440 USING P4 AS CASH_1, MODEL [C]LINE GLOBAL.440 USING P6 AS CASH_2, MODEL [C]LINE GLOBAL.440 USING P8 AS CASH_3, MODEL [C]LINE GLOBAL.440 USING P10 AS CASH_4, MODEL [C]LINE GLOBAL.440 USING P12 AS CASH_5, MODEL [C]LINE GLOBAL.440 USING P14 AS CASH_6, MODEL [C]LINE GLOBAL.440 USING P16 AS CASH_7, MODEL [C]LINE GLOBAL.440 USING P18 AS CASH_8, MODEL [C]LINE GLOBAL.440 USING P20 AS CASH_9, MODEL [C]LINE GLOBAL.412 USING P2 AS TOAS_20, MODEL [C]LINE GLOBAL.412 USING P4 AS TOAS_21, MODEL [C]LINE GLOBAL.412 USING P6 AS TOAS_22, MODEL [C]LINE GLOBAL.412 USING P8 AS TOAS_23, MODEL [C]LINE GLOBAL.412 USING P10 AS TOAS_24, MODEL [C]LINE GLOBAL.412 USING P12 AS TOAS_25, MODEL [C]LINE GLOBAL.412 USING P14 AS TOAS_26, MODEL [C]LINE GLOBAL.412 USING P16 AS TOAS_27, MODEL [C]LINE GLOBAL.412 USING P18 AS TOAS_28, MODEL [C]LINE GLOBAL.412 USING P20 AS TOAS_29, MODEL [C]LINE GLOBAL.413 USING P2 AS SHFD_20, MODEL [C]LINE GLOBAL.413 USING P4 AS SHFD_21, MODEL [C]LINE GLOBAL.413 USING P6 AS SHFD_22, MODEL [C]LINE GLOBAL.413 USING P8 AS SHFD_23, MODEL [C]LINE GLOBAL.413 USING P10 AS SHFD_24, MODEL [C]LINE GLOBAL.413 USING P12 AS SHFD_25, MODEL [C]LINE GLOBAL.413 USING P14 AS SHFD_26, MODEL [C]LINE GLOBAL.413 USING P16 AS SHFD_27, MODEL [C]LINE GLOBAL.413 USING P18 AS SHFD_28, MODEL [C]LINE GLOBAL.413 USING P20 AS SHFD_29, MODEL [C]LINE GLOBAL.414 USING P2 AS CAPI, MODEL [C]LINE GLOBAL.414 USING P4 AS CAPI_1, MODEL [C]LINE GLOBAL.414 USING P6 AS CAPI_2, MODEL [C]LINE GLOBAL.414 USING P8 AS CAPI_3, MODEL [C]LINE GLOBAL.414 USING P10 AS CAPI_4, MODEL [C]LINE GLOBAL.414 USING P12 AS CAPI_5, MODEL [C]LINE GLOBAL.414 USING P14 AS CAPI_6, MODEL [C]LINE GLOBAL.414 USING P16 AS CAPI_7, MODEL [C]LINE GLOBAL.414 USING P18 AS CAPI_8, MODEL [C]LINE GLOBAL.414 USING P20 AS CAPI_9, MODEL [C]LINE GLOBAL.415 USING P2 AS OSFD, MODEL [C]LINE GLOBAL.415 USING P4 AS OSFD_1, MODEL [C]LINE GLOBAL.415 USING P6 AS OSFD_2, MODEL [C]LINE GLOBAL.415 USING P8 AS OSFD_3, MODEL [C]LINE GLOBAL.415 USING P10 AS OSFD_4, MODEL [C]LINE GLOBAL.415 USING P12 AS OSFD_5, MODEL [C]LINE GLOBAL.415 USING P14 AS OSFD_6, MODEL [C]LINE GLOBAL.415 USING P16 AS OSFD_7, MODEL [C]LINE GLOBAL.415 USING P18 AS OSFD_8, MODEL [C]LINE GLOBAL.415 USING P20 AS OSFD_9, MODEL [C]LINE GLOBAL.416 USING P2 AS NCLI, MODEL [C]LINE GLOBAL.416 USING P4 AS NCLI_1, MODEL [C]LINE GLOBAL.416 USING P6 AS NCLI_2, MODEL [C]LINE GLOBAL.416 USING P8 AS NCLI_3, MODEL [C]LINE GLOBAL.416 USING P10 AS NCLI_4, MODEL [C]LINE GLOBAL.416 USING P12 AS NCLI_5, MODEL [C]LINE GLOBAL.416 USING P14 AS NCLI_6, MODEL [C]LINE GLOBAL.416 USING P16 AS NCLI_7, MODEL [C]LINE GLOBAL.416 USING P18 AS NCLI_8, MODEL [C]LINE GLOBAL.416 USING P20 AS NCLI_9, MODEL [C]LINE GLOBAL.417 USING P2 AS LTDB, MODEL [C]LINE GLOBAL.417 USING P4 AS LTDB_1, MODEL [C]LINE GLOBAL.417 USING P6 AS LTDB_2, MODEL [C]LINE GLOBAL.417 USING P8 AS LTDB_3, MODEL [C]LINE GLOBAL.417 USING P10 AS LTDB_4, MODEL [C]LINE GLOBAL.417 USING P12 AS LTDB_5, MODEL [C]LINE GLOBAL.417 USING P14 AS LTDB_6, MODEL [C]LINE GLOBAL.417 USING P16 AS LTDB_7, MODEL [C]LINE GLOBAL.417 USING P18 AS LTDB_8, MODEL [C]LINE GLOBAL.417 USING P20 AS LTDB_9, MODEL [C]LINE GLOBAL.418 USING P2 AS ONCL, MODEL [C]LINE GLOBAL.418 USING P4 AS ONCL_1, MODEL [C]LINE GLOBAL.418 USING P6 AS ONCL_2, MODEL [C]LINE GLOBAL.418 USING P8 AS ONCL_3, MODEL [C]LINE GLOBAL.418 USING P10 AS ONCL_4, MODEL [C]LINE GLOBAL.418 USING P12 AS ONCL_5, MODEL [C]LINE GLOBAL.418 USING P14 AS ONCL_6, MODEL [C]LINE GLOBAL.418 USING P16 AS ONCL_7, MODEL [C]LINE GLOBAL.418 USING P18 AS ONCL_8, MODEL [C]LINE GLOBAL.418 USING P20 AS ONCL_9, MODEL [C]LINE GLOBAL.453 USING P2 AS PROV, MODEL [C]LINE GLOBAL.453 USING P4 AS PROV_1, MODEL [C]LINE GLOBAL.453 USING P6 AS PROV_2, MODEL [C]LINE GLOBAL.453 USING P8 AS PROV_3, MODEL [C]LINE GLOBAL.453 USING P10 AS PROV_4, MODEL [C]LINE GLOBAL.453 USING P12 AS PROV_5, MODEL [C]LINE GLOBAL.453 USING P14 AS PROV_6, MODEL [C]LINE GLOBAL.453 USING P16 AS PROV_7, MODEL [C]LINE GLOBAL.453 USING P18 AS PROV_8, MODEL [C]LINE GLOBAL.453 USING P20 AS PROV_9, MODEL [C]LINE GLOBAL.419 USING P2 AS CULI, MODEL [C]LINE GLOBAL.419 USING P4 AS CULI_1, MODEL [C]LINE GLOBAL.419 USING P6 AS CULI_2, MODEL [C]LINE GLOBAL.419 USING P8 AS CULI_3, MODEL [C]LINE GLOBAL.419 USING P10 AS CULI_4, MODEL [C]LINE GLOBAL.419 USING P12 AS CULI_5, MODEL [C]LINE GLOBAL.419 USING P14 AS CULI_6, MODEL [C]LINE GLOBAL.419 USING P16 AS CULI_7, MODEL [C]LINE GLOBAL.419 USING P18 AS CULI_8, MODEL [C]LINE GLOBAL.419 USING P20 AS CULI_9, MODEL [C]LINE GLOBAL.420 USING P2 AS LOAN, MODEL [C]LINE GLOBAL.420 USING P4 AS LOAN_1, MODEL [C]LINE GLOBAL.420 USING P6 AS LOAN_2, MODEL [C]LINE GLOBAL.420 USING P8 AS LOAN_3, MODEL [C]LINE GLOBAL.420 USING P10 AS LOAN_4, MODEL [C]LINE GLOBAL.420 USING P12 AS LOAN_5, MODEL [C]LINE GLOBAL.420 USING P14 AS LOAN_6, MODEL [C]LINE GLOBAL.420 USING P16 AS LOAN_7, MODEL [C]LINE GLOBAL.420 USING P18 AS LOAN_8, MODEL [C]LINE GLOBAL.420 USING P20 AS LOAN_9, MODEL [C]LINE GLOBAL.421 USING P2 AS CRED, MODEL [C]LINE GLOBAL.421 USING P4 AS CRED_1, MODEL [C]LINE GLOBAL.421 USING P6 AS CRED_2, MODEL [C]LINE GLOBAL.421 USING P8 AS CRED_3, MODEL [C]LINE GLOBAL.421 USING P10 AS CRED_4, MODEL [C]LINE GLOBAL.421 USING P12 AS CRED_5, MODEL [C]LINE GLOBAL.421 USING P14 AS CRED_6, MODEL [C]LINE GLOBAL.421 USING P16 AS CRED_7, MODEL [C]LINE GLOBAL.421 USING P18 AS CRED_8, MODEL [C]LINE GLOBAL.421 USING P20 AS CRED_9, MODEL [C]LINE GLOBAL.422 USING P2 AS OCLI, MODEL [C]LINE GLOBAL.422 USING P4 AS OCLI_1, MODEL [C]LINE GLOBAL.422 USING P6 AS OCLI_2, MODEL [C]LINE GLOBAL.422 USING P8 AS OCLI_3, MODEL [C]LINE GLOBAL.422 USING P10 AS OCLI_4, MODEL [C]LINE GLOBAL.422 USING P12 AS OCLI_5, MODEL [C]LINE GLOBAL.422 USING P14 AS OCLI_6, MODEL [C]LINE GLOBAL.422 USING P16 AS OCLI_7, MODEL [C]LINE GLOBAL.422 USING P18 AS OCLI_8, MODEL [C]LINE GLOBAL.422 USING P20 AS OCLI_9, MODEL [C]LINE GLOBAL.423 USING P2 AS TSHF, MODEL [C]LINE GLOBAL.423 USING P4 AS TSHF_1, MODEL [C]LINE GLOBAL.423 USING P6 AS TSHF_2, MODEL [C]LINE GLOBAL.423 USING P8 AS TSHF_3, MODEL [C]LINE GLOBAL.423 USING P10 AS TSHF_4, MODEL [C]LINE GLOBAL.423 USING P12 AS TSHF_5, MODEL [C]LINE GLOBAL.423 USING P14 AS TSHF_6, MODEL [C]LINE GLOBAL.423 USING P16 AS TSHF_7, MODEL [C]LINE GLOBAL.423 USING P18 AS TSHF_8, MODEL [C]LINE GLOBAL.423 USING P20 AS TSHF_9, MODEL [C]LINE GLOBAL.424 USING P2 AS WKCA, MODEL [C]LINE GLOBAL.424 USING P4 AS WKCA_1, MODEL [C]LINE GLOBAL.424 USING P6 AS WKCA_2, MODEL [C]LINE GLOBAL.424 USING P8 AS WKCA_3, MODEL [C]LINE GLOBAL.424 USING P10 AS WKCA_4, MODEL [C]LINE GLOBAL.424 USING P12 AS WKCA_5, MODEL [C]LINE GLOBAL.424 USING P14 AS WKCA_6, MODEL [C]LINE GLOBAL.424 USING P16 AS WKCA_7, MODEL [C]LINE GLOBAL.424 USING P18 AS WKCA_8, MODEL [C]LINE GLOBAL.424 USING P20 AS WKCA_9, MODEL [C]LINE GLOBAL.452 USING P2 AS NCAS, MODEL [C]LINE GLOBAL.452 USING P4 AS NCAS_1, MODEL [C]LINE GLOBAL.452 USING P6 AS NCAS_2, MODEL [C]LINE GLOBAL.452 USING P8 AS NCAS_3, MODEL [C]LINE GLOBAL.452 USING P10 AS NCAS_4, MODEL [C]LINE GLOBAL.452 USING P12 AS NCAS_5, MODEL [C]LINE GLOBAL.452 USING P14 AS NCAS_6, MODEL [C]LINE GLOBAL.452 USING P16 AS NCAS_7, MODEL [C]LINE GLOBAL.452 USING P18 AS NCAS_8, MODEL [C]LINE GLOBAL.452 USING P20 AS NCAS_9, MODEL [C]LINE GLOBAL.450 USING P2 AS ENVA, MODEL [C]LINE GLOBAL.450 USING P4 AS ENVA_1, MODEL [C]LINE GLOBAL.450 USING P6 AS ENVA_2, MODEL [C]LINE GLOBAL.450 USING P8 AS ENVA_3, MODEL [C]LINE GLOBAL.450 USING P10 AS ENVA_4, MODEL [C]LINE GLOBAL.450 USING P12 AS ENVA_5, MODEL [C]LINE GLOBAL.450 USING P14 AS ENVA_6, MODEL [C]LINE GLOBAL.450 USING P16 AS ENVA_7, MODEL [C]LINE GLOBAL.450 USING P18 AS ENVA_8, MODEL [C]LINE GLOBAL.450 USING P20 AS ENVA_9, MODEL [C]LINE GLOBAL.425 USING P21 AS EMPL_10, MODEL [C]LINE GLOBAL.448 USING P2 AS OPRE_20, MODEL [C]LINE GLOBAL.448 USING P4 AS OPRE_21, MODEL [C]LINE GLOBAL.448 USING P6 AS OPRE_22, MODEL [C]LINE GLOBAL.448 USING P8 AS OPRE_23, MODEL [C]LINE GLOBAL.448 USING P10 AS OPRE_24, MODEL [C]LINE GLOBAL.448 USING P12 AS OPRE_25, MODEL [C]LINE GLOBAL.448 USING P14 AS OPRE_26, MODEL [C]LINE GLOBAL.448 USING P16 AS OPRE_27, MODEL [C]LINE GLOBAL.448 USING P18 AS OPRE_28, MODEL [C]LINE GLOBAL.448 USING P20 AS OPRE_29, MODEL [C]LINE GLOBAL.426 USING P2 AS TURN, MODEL [C]LINE GLOBAL.426 USING P4 AS TURN_1, MODEL [C]LINE GLOBAL.426 USING P6 AS TURN_2, MODEL [C]LINE GLOBAL.426 USING P8 AS TURN_3, MODEL [C]LINE GLOBAL.426 USING P10 AS TURN_4, MODEL [C]LINE GLOBAL.426 USING P12 AS TURN_5, MODEL [C]LINE GLOBAL.426 USING P14 AS TURN_6, MODEL [C]LINE GLOBAL.426 USING P16 AS TURN_7, MODEL [C]LINE GLOBAL.426 USING P18 AS TURN_8, MODEL [C]LINE GLOBAL.426 USING P20 AS TURN_9, MODEL [C]LINE GLOBAL.441 USING P2 AS COST, MODEL [C]LINE GLOBAL.441 USING P4 AS COST_1, MODEL [C]LINE GLOBAL.441 USING P6 AS COST_2, MODEL [C]LINE GLOBAL.441 USING P8 AS COST_3, MODEL [C]LINE GLOBAL.441 USING P10 AS COST_4, MODEL [C]LINE GLOBAL.441 USING P12 AS COST_5, MODEL [C]LINE GLOBAL.441 USING P14 AS COST_6, MODEL [C]LINE GLOBAL.441 USING P16 AS COST_7, MODEL [C]LINE GLOBAL.441 USING P18 AS COST_8, MODEL [C]LINE GLOBAL.441 USING P20 AS COST_9, MODEL [C]LINE GLOBAL.442 USING P2 AS GROS, MODEL [C]LINE GLOBAL.442 USING P4 AS GROS_1, MODEL [C]LINE GLOBAL.442 USING P6 AS GROS_2, MODEL [C]LINE GLOBAL.442 USING P8 AS GROS_3, MODEL [C]LINE GLOBAL.442 USING P10 AS GROS_4, MODEL [C]LINE GLOBAL.442 USING P12 AS GROS_5, MODEL [C]LINE GLOBAL.442 USING P14 AS GROS_6, MODEL [C]LINE GLOBAL.442 USING P16 AS GROS_7, MODEL [C]LINE GLOBAL.442 USING P18 AS GROS_8, MODEL [C]LINE GLOBAL.442 USING P20 AS GROS_9, MODEL [C]LINE GLOBAL.443 USING P2 AS OOPE, MODEL [C]LINE GLOBAL.443 USING P4 AS OOPE_1, MODEL [C]LINE GLOBAL.443 USING P6 AS OOPE_2, MODEL [C]LINE GLOBAL.443 USING P8 AS OOPE_3, MODEL [C]LINE GLOBAL.443 USING P10 AS OOPE_4, MODEL [C]LINE GLOBAL.443 USING P12 AS OOPE_5, MODEL [C]LINE GLOBAL.443 USING P14 AS OOPE_6, MODEL [C]LINE GLOBAL.443 USING P16 AS OOPE_7, MODEL [C]LINE GLOBAL.443 USING P18 AS OOPE_8, MODEL [C]LINE GLOBAL.443 USING P20 AS OOPE_9, MODEL [C]LINE GLOBAL.427 USING P2 AS OPPL, MODEL [C]LINE GLOBAL.427 USING P4 AS OPPL_1, MODEL [C]LINE GLOBAL.427 USING P6 AS OPPL_2, MODEL [C]LINE GLOBAL.427 USING P8 AS OPPL_3, MODEL [C]LINE GLOBAL.427 USING P10 AS OPPL_4, MODEL [C]LINE GLOBAL.427 USING P12 AS OPPL_5, MODEL [C]LINE GLOBAL.427 USING P14 AS OPPL_6, MODEL [C]LINE GLOBAL.427 USING P16 AS OPPL_7, MODEL [C]LINE GLOBAL.427 USING P18 AS OPPL_8, MODEL [C]LINE GLOBAL.427 USING P20 AS OPPL_9, MODEL [C]LINE GLOBAL.444 USING P2 AS FIRE, MODEL [C]LINE GLOBAL.444 USING P4 AS FIRE_1, MODEL [C]LINE GLOBAL.444 USING P6 AS FIRE_2, MODEL [C]LINE GLOBAL.444 USING P8 AS FIRE_3, MODEL [C]LINE GLOBAL.444 USING P10 AS FIRE_4, MODEL [C]LINE GLOBAL.444 USING P12 AS FIRE_5, MODEL [C]LINE GLOBAL.444 USING P14 AS FIRE_6, MODEL [C]LINE GLOBAL.444 USING P16 AS FIRE_7, MODEL [C]LINE GLOBAL.444 USING P18 AS FIRE_8, MODEL [C]LINE GLOBAL.444 USING P20 AS FIRE_9, MODEL [C]LINE GLOBAL.445 USING P2 AS FIEX, MODEL [C]LINE GLOBAL.445 USING P4 AS FIEX_1, MODEL [C]LINE GLOBAL.445 USING P6 AS FIEX_2, MODEL [C]LINE GLOBAL.445 USING P8 AS FIEX_3, MODEL [C]LINE GLOBAL.445 USING P10 AS FIEX_4, MODEL [C]LINE GLOBAL.445 USING P12 AS FIEX_5, MODEL [C]LINE GLOBAL.445 USING P14 AS FIEX_6, MODEL [C]LINE GLOBAL.445 USING P16 AS FIEX_7, MODEL [C]LINE GLOBAL.445 USING P18 AS FIEX_8, MODEL [C]LINE GLOBAL.445 USING P20 AS FIEX_9, MODEL [C]LINE GLOBAL.428 USING P2 AS FIPL, MODEL [C]LINE GLOBAL.428 USING P4 AS FIPL_1, MODEL [C]LINE GLOBAL.428 USING P6 AS FIPL_2, MODEL [C]LINE GLOBAL.428 USING P8 AS FIPL_3, MODEL [C]LINE GLOBAL.428 USING P10 AS FIPL_4, MODEL [C]LINE GLOBAL.428 USING P12 AS FIPL_5, MODEL [C]LINE GLOBAL.428 USING P14 AS FIPL_6, MODEL [C]LINE GLOBAL.428 USING P16 AS FIPL_7, MODEL [C]LINE GLOBAL.428 USING P18 AS FIPL_8, MODEL [C]LINE GLOBAL.428 USING P20 AS FIPL_9, MODEL [C]LINE GLOBAL.429 USING P2 AS PLBT_20, MODEL [C]LINE GLOBAL.429 USING P4 AS PLBT_21, MODEL [C]LINE GLOBAL.429 USING P6 AS PLBT_22, MODEL [C]LINE GLOBAL.429 USING P8 AS PLBT_23, MODEL [C]LINE GLOBAL.429 USING P10 AS PLBT_24, MODEL [C]LINE GLOBAL.429 USING P12 AS PLBT_25, MODEL [C]LINE GLOBAL.429 USING P14 AS PLBT_26, MODEL [C]LINE GLOBAL.429 USING P16 AS PLBT_27, MODEL [C]LINE GLOBAL.429 USING P18 AS PLBT_28, MODEL [C]LINE GLOBAL.429 USING P20 AS PLBT_29, MODEL [C]LINE GLOBAL.430 USING P2 AS TAXA, MODEL [C]LINE GLOBAL.430 USING P4 AS TAXA_1, MODEL [C]LINE GLOBAL.430 USING P6 AS TAXA_2, MODEL [C]LINE GLOBAL.430 USING P8 AS TAXA_3, MODEL [C]LINE GLOBAL.430 USING P10 AS TAXA_4, MODEL [C]LINE GLOBAL.430 USING P12 AS TAXA_5, MODEL [C]LINE GLOBAL.430 USING P14 AS TAXA_6, MODEL [C]LINE GLOBAL.430 USING P16 AS TAXA_7, MODEL [C]LINE GLOBAL.430 USING P18 AS TAXA_8, MODEL [C]LINE GLOBAL.430 USING P20 AS TAXA_9, MODEL [C]LINE GLOBAL.431 USING P2 AS PLAT, MODEL [C]LINE GLOBAL.431 USING P4 AS PLAT_1, MODEL [C]LINE GLOBAL.431 USING P6 AS PLAT_2, MODEL [C]LINE GLOBAL.431 USING P8 AS PLAT_3, MODEL [C]LINE GLOBAL.431 USING P10 AS PLAT_4, MODEL [C]LINE GLOBAL.431 USING P12 AS PLAT_5, MODEL [C]LINE GLOBAL.431 USING P14 AS PLAT_6, MODEL [C]LINE GLOBAL.431 USING P16 AS PLAT_7, MODEL [C]LINE GLOBAL.431 USING P18 AS PLAT_8, MODEL [C]LINE GLOBAL.431 USING P20 AS PLAT_9, MODEL [C]LINE GLOBAL.446 USING P2 AS EXRE, MODEL [C]LINE GLOBAL.446 USING P4 AS EXRE_1, MODEL [C]LINE GLOBAL.446 USING P6 AS EXRE_2, MODEL [C]LINE GLOBAL.446 USING P8 AS EXRE_3, MODEL [C]LINE GLOBAL.446 USING P10 AS EXRE_4, MODEL [C]LINE GLOBAL.446 USING P12 AS EXRE_5, MODEL [C]LINE GLOBAL.446 USING P14 AS EXRE_6, MODEL [C]LINE GLOBAL.446 USING P16 AS EXRE_7, MODEL [C]LINE GLOBAL.446 USING P18 AS EXRE_8, MODEL [C]LINE GLOBAL.446 USING P20 AS EXRE_9, MODEL [C]LINE GLOBAL.447 USING P2 AS EXEX, MODEL [C]LINE GLOBAL.447 USING P4 AS EXEX_1, MODEL [C]LINE GLOBAL.447 USING P6 AS EXEX_2, MODEL [C]LINE GLOBAL.447 USING P8 AS EXEX_3, MODEL [C]LINE GLOBAL.447 USING P10 AS EXEX_4, MODEL [C]LINE GLOBAL.447 USING P12 AS EXEX_5, MODEL [C]LINE GLOBAL.447 USING P14 AS EXEX_6, MODEL [C]LINE GLOBAL.447 USING P16 AS EXEX_7, MODEL [C]LINE GLOBAL.447 USING P18 AS EXEX_8, MODEL [C]LINE GLOBAL.447 USING P20 AS EXEX_9, MODEL [C]LINE GLOBAL.432 USING P2 AS EXTR, MODEL [C]LINE GLOBAL.432 USING P4 AS EXTR_1, MODEL [C]LINE GLOBAL.432 USING P6 AS EXTR_2, MODEL [C]LINE GLOBAL.432 USING P8 AS EXTR_3, MODEL [C]LINE GLOBAL.432 USING P10 AS EXTR_4, MODEL [C]LINE GLOBAL.432 USING P12 AS EXTR_5, MODEL [C]LINE GLOBAL.432 USING P14 AS EXTR_6, MODEL [C]LINE GLOBAL.432 USING P16 AS EXTR_7, MODEL [C]LINE GLOBAL.432 USING P18 AS EXTR_8, MODEL [C]LINE GLOBAL.432 USING P20 AS EXTR_9, MODEL [C]LINE GLOBAL.433 USING P2 AS PL_20, MODEL [C]LINE GLOBAL.433 USING P4 AS PL_21, MODEL [C]LINE GLOBAL.433 USING P6 AS PL_22, MODEL [C]LINE GLOBAL.433 USING P8 AS PL_23, MODEL [C]LINE GLOBAL.433 USING P10 AS PL_24, MODEL [C]LINE GLOBAL.433 USING P12 AS PL_25, MODEL [C]LINE GLOBAL.433 USING P14 AS PL_26, MODEL [C]LINE GLOBAL.433 USING P16 AS PL_27, MODEL [C]LINE GLOBAL.433 USING P18 AS PL_28, MODEL [C]LINE GLOBAL.433 USING P20 AS PL_29, MODEL [C]LINE GLOBAL.451 USING P2 AS EXPT, MODEL [C]LINE GLOBAL.451 USING P4 AS EXPT_1, MODEL [C]LINE GLOBAL.451 USING P6 AS EXPT_2, MODEL [C]LINE GLOBAL.451 USING P8 AS EXPT_3, MODEL [C]LINE GLOBAL.451 USING P10 AS EXPT_4, MODEL [C]LINE GLOBAL.451 USING P12 AS EXPT_5, MODEL [C]LINE GLOBAL.451 USING P14 AS EXPT_6, MODEL [C]LINE GLOBAL.451 USING P16 AS EXPT_7, MODEL [C]LINE GLOBAL.451 USING P18 AS EXPT_8, MODEL [C]LINE GLOBAL.451 USING P20 AS EXPT_9, MODEL [C]LINE GLOBAL.434 USING P2 AS MATE, MODEL [C]LINE GLOBAL.434 USING P4 AS MATE_1, MODEL [C]LINE GLOBAL.434 USING P6 AS MATE_2, MODEL [C]LINE GLOBAL.434 USING P8 AS MATE_3, MODEL [C]LINE GLOBAL.434 USING P10 AS MATE_4, MODEL [C]LINE GLOBAL.434 USING P12 AS MATE_5, MODEL [C]LINE GLOBAL.434 USING P14 AS MATE_6, MODEL [C]LINE GLOBAL.434 USING P16 AS MATE_7, MODEL [C]LINE GLOBAL.434 USING P18 AS MATE_8, MODEL [C]LINE GLOBAL.434 USING P20 AS MATE_9, MODEL [C]LINE GLOBAL.435 USING P2 AS STAF, MODEL [C]LINE GLOBAL.435 USING P4 AS STAF_1, MODEL [C]LINE GLOBAL.435 USING P6 AS STAF_2, MODEL [C]LINE GLOBAL.435 USING P8 AS STAF_3, MODEL [C]LINE GLOBAL.435 USING P10 AS STAF_4, MODEL [C]LINE GLOBAL.435 USING P12 AS STAF_5, MODEL [C]LINE GLOBAL.435 USING P14 AS STAF_6, MODEL [C]LINE GLOBAL.435 USING P16 AS STAF_7, MODEL [C]LINE GLOBAL.435 USING P18 AS STAF_8, MODEL [C]LINE GLOBAL.435 USING P20 AS STAF_9, MODEL [C]LINE GLOBAL.436 USING P2 AS DEPR, MODEL [C]LINE GLOBAL.436 USING P4 AS DEPR_1, MODEL [C]LINE GLOBAL.436 USING P6 AS DEPR_2, MODEL [C]LINE GLOBAL.436 USING P8 AS DEPR_3, MODEL [C]LINE GLOBAL.436 USING P10 AS DEPR_4, MODEL [C]LINE GLOBAL.436 USING P12 AS DEPR_5, MODEL [C]LINE GLOBAL.436 USING P14 AS DEPR_6, MODEL [C]LINE GLOBAL.436 USING P16 AS DEPR_7, MODEL [C]LINE GLOBAL.436 USING P18 AS DEPR_8, MODEL [C]LINE GLOBAL.436 USING P20 AS DEPR_9, MODEL [C]LINE GLOBAL.457 USING P2 AS OOPI, MODEL [C]LINE GLOBAL.457 USING P4 AS OOPI_1, MODEL [C]LINE GLOBAL.457 USING P6 AS OOPI_2, MODEL [C]LINE GLOBAL.457 USING P8 AS OOPI_3, MODEL [C]LINE GLOBAL.457 USING P10 AS OOPI_4, MODEL [C]LINE GLOBAL.457 USING P12 AS OOPI_5, MODEL [C]LINE GLOBAL.457 USING P14 AS OOPI_6, MODEL [C]LINE GLOBAL.457 USING P16 AS OOPI_7, MODEL [C]LINE GLOBAL.457 USING P18 AS OOPI_8, MODEL [C]LINE GLOBAL.457 USING P20 AS OOPI_9, MODEL [C]LINE GLOBAL.437 USING P2 AS INTE, MODEL [C]LINE GLOBAL.437 USING P4 AS INTE_1, MODEL [C]LINE GLOBAL.437 USING P6 AS INTE_2, MODEL [C]LINE GLOBAL.437 USING P8 AS INTE_3, MODEL [C]LINE GLOBAL.437 USING P10 AS INTE_4, MODEL [C]LINE GLOBAL.437 USING P12 AS INTE_5, MODEL [C]LINE GLOBAL.437 USING P14 AS INTE_6, MODEL [C]LINE GLOBAL.437 USING P16 AS INTE_7, MODEL [C]LINE GLOBAL.437 USING P18 AS INTE_8, MODEL [C]LINE GLOBAL.437 USING P20 AS INTE_9, MODEL [C]LINE GLOBAL.454 USING P2 AS RD, MODEL [C]LINE GLOBAL.454 USING P4 AS RD_1, MODEL [C]LINE GLOBAL.454 USING P6 AS RD_2, MODEL [C]LINE GLOBAL.454 USING P8 AS RD_3, MODEL [C]LINE GLOBAL.454 USING P10 AS RD_4, MODEL [C]LINE GLOBAL.454 USING P12 AS RD_5, MODEL [C]LINE GLOBAL.454 USING P14 AS RD_6, MODEL [C]LINE GLOBAL.454 USING P16 AS RD_7, MODEL [C]LINE GLOBAL.454 USING P18 AS RD_8, MODEL [C]LINE GLOBAL.454 USING P20 AS RD_9, MODEL [C]LINE GLOBAL.438 USING P2 AS CF_20, MODEL [C]LINE GLOBAL.438 USING P4 AS CF_21, MODEL [C]LINE GLOBAL.438 USING P6 AS CF_22, MODEL [C]LINE GLOBAL.438 USING P8 AS CF_23, MODEL [C]LINE GLOBAL.438 USING P10 AS CF_24, MODEL [C]LINE GLOBAL.438 USING P12 AS CF_25, MODEL [C]LINE GLOBAL.438 USING P14 AS CF_26, MODEL [C]LINE GLOBAL.438 USING P16 AS CF_27, MODEL [C]LINE GLOBAL.438 USING P18 AS CF_28, MODEL [C]LINE GLOBAL.438 USING P20 AS CF_29, MODEL [C]LINE GLOBAL.439 USING P2 AS AV, MODEL [C]LINE GLOBAL.439 USING P4 AS AV_1, MODEL [C]LINE GLOBAL.439 USING P6 AS AV_2, MODEL [C]LINE GLOBAL.439 USING P8 AS AV_3, MODEL [C]LINE GLOBAL.439 USING P10 AS AV_4, MODEL [C]LINE GLOBAL.439 USING P12 AS AV_5, MODEL [C]LINE GLOBAL.439 USING P14 AS AV_6, MODEL [C]LINE GLOBAL.439 USING P16 AS AV_7, MODEL [C]LINE GLOBAL.439 USING P18 AS AV_8, MODEL [C]LINE GLOBAL.439 USING P20 AS AV_9, MODEL [C]LINE GLOBAL.449 USING P2 AS EBTA, MODEL [C]LINE GLOBAL.449 USING P4 AS EBTA_1, MODEL [C]LINE GLOBAL.449 USING P6 AS EBTA_2, MODEL [C]LINE GLOBAL.449 USING P8 AS EBTA_3, MODEL [C]LINE GLOBAL.449 USING P10 AS EBTA_4, MODEL [C]LINE GLOBAL.449 USING P12 AS EBTA_5, MODEL [C]LINE GLOBAL.449 USING P14 AS EBTA_6, MODEL [C]LINE GLOBAL.449 USING P16 AS EBTA_7, MODEL [C]LINE GLOBAL.449 USING P18 AS EBTA_8, MODEL [C]LINE GLOBAL.449 USING P20 AS EBTA_9, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P2 AS 313061, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P4 AS 313061_1, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P6 AS 313061_2, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P8 AS 313061_3, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P10 AS 313061_4, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P12 AS 313061_5, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P14 AS 313061_6, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P16 AS 313061_7, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P18 AS 313061_8, MODEL [C]LINE DETAILLED_BALSHT.313061 USING P20 AS 313061_9, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P2 AS 320010, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P4 AS 320010_1, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P6 AS 320010_2, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P8 AS 320010_3, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P10 AS 320010_4, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P12 AS 320010_5, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P14 AS 320010_6, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P16 AS 320010_7, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P18 AS 320010_8, MODEL [C]LINE DETAILLED_BALSHT.320010 USING P20 AS 320010_9, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P2 AS 320015, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P4 AS 320015_1, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P6 AS 320015_2, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P8 AS 320015_3, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P10 AS 320015_4, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P12 AS 320015_5, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P14 AS 320015_6, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P16 AS 320015_7, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P18 AS 320015_8, MODEL [C]LINE DETAILLED_BALSHT.320015 USING P20 AS 320015_9, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P2 AS 320020, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P4 AS 320020_1, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P6 AS 320020_2, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P8 AS 320020_3, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P10 AS 320020_4, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P12 AS 320020_5, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P14 AS 320020_6, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P16 AS 320020_7, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P18 AS 320020_8, MODEL [C]LINE DETAILLED_BALSHT.320020 USING P20 AS 320020_9, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P2 AS 320025, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P4 AS 320025_1, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P6 AS 320025_2, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P8 AS 320025_3, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P10 AS 320025_4, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P12 AS 320025_5, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P14 AS 320025_6, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P16 AS 320025_7, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P18 AS 320025_8, MODEL [C]LINE DETAILLED_BALSHT.320025 USING P20 AS 320025_9, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P2 AS 320030, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P4 AS 320030_1, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P6 AS 320030_2, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P8 AS 320030_3, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P10 AS 320030_4, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P12 AS 320030_5, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P14 AS 320030_6, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P16 AS 320030_7, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P18 AS 320030_8, MODEL [C]LINE DETAILLED_BALSHT.320030 USING P20 AS 320030_9, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P2 AS 313052, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P4 AS 313052_1, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P6 AS 313052_2, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P8 AS 313052_3, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P10 AS 313052_4, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P12 AS 313052_5, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P14 AS 313052_6, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P16 AS 313052_7, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P18 AS 313052_8, MODEL [C]LINE DETAILLED_BALSHT.313052 USING P20 AS 313052_9, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P2 AS 320040, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P4 AS 320040_1, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P6 AS 320040_2, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P8 AS 320040_3, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P10 AS 320040_4, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P12 AS 320040_5, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P14 AS 320040_6, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P16 AS 320040_7, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P18 AS 320040_8, MODEL [C]LINE DETAILLED_BALSHT.320040 USING P20 AS 320040_9, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P2 AS 320045, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P4 AS 320045_1, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P6 AS 320045_2, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P8 AS 320045_3, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P10 AS 320045_4, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P12 AS 320045_5, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P14 AS 320045_6, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P16 AS 320045_7, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P18 AS 320045_8, MODEL [C]LINE DETAILLED_BALSHT.320045 USING P20 AS 320045_9, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P2 AS 320050, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P4 AS 320050_1, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P6 AS 320050_2, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P8 AS 320050_3, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P10 AS 320050_4, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P12 AS 320050_5, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P14 AS 320050_6, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P16 AS 320050_7, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P18 AS 320050_8, MODEL [C]LINE DETAILLED_BALSHT.320050 USING P20 AS 320050_9, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P2 AS 320055, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P4 AS 320055_1, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P6 AS 320055_2, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P8 AS 320055_3, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P10 AS 320055_4, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P12 AS 320055_5, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P14 AS 320055_6, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P16 AS 320055_7, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P18 AS 320055_8, MODEL [C]LINE DETAILLED_BALSHT.320055 USING P20 AS 320055_9, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P2 AS 313054, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P4 AS 313054_1, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P6 AS 313054_2, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P8 AS 313054_3, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P10 AS 313054_4, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P12 AS 313054_5, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P14 AS 313054_6, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P16 AS 313054_7, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P18 AS 313054_8, MODEL [C]LINE DETAILLED_BALSHT.313054 USING P20 AS 313054_9, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P2 AS 320065, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P4 AS 320065_1, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P6 AS 320065_2, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P8 AS 320065_3, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P10 AS 320065_4, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P12 AS 320065_5, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P14 AS 320065_6, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P16 AS 320065_7, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P18 AS 320065_8, MODEL [C]LINE DETAILLED_BALSHT.320065 USING P20 AS 320065_9, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P2 AS 320070, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P4 AS 320070_1, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P6 AS 320070_2, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P8 AS 320070_3, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P10 AS 320070_4, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P12 AS 320070_5, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P14 AS 320070_6, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P16 AS 320070_7, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P18 AS 320070_8, MODEL [C]LINE DETAILLED_BALSHT.320070 USING P20 AS 320070_9, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P2 AS 313050, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P4 AS 313050_1, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P6 AS 313050_2, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P8 AS 313050_3, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P10 AS 313050_4, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P12 AS 313050_5, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P14 AS 313050_6, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P16 AS 313050_7, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P18 AS 313050_8, MODEL [C]LINE DETAILLED_BALSHT.313050 USING P20 AS 313050_9, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P2 AS 313051, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P4 AS 313051_1, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P6 AS 313051_2, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P8 AS 313051_3, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P10 AS 313051_4, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P12 AS 313051_5, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P14 AS 313051_6, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P16 AS 313051_7, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P18 AS 313051_8, MODEL [C]LINE DETAILLED_BALSHT.313051 USING P20 AS 313051_9, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P2 AS 320085, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P4 AS 320085_1, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P6 AS 320085_2, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P8 AS 320085_3, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P10 AS 320085_4, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P12 AS 320085_5, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P14 AS 320085_6, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P16 AS 320085_7, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P18 AS 320085_8, MODEL [C]LINE DETAILLED_BALSHT.320085 USING P20 AS 320085_9, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P2 AS 313068, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P4 AS 313068_1, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P6 AS 313068_2, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P8 AS 313068_3, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P10 AS 313068_4, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P12 AS 313068_5, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P14 AS 313068_6, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P16 AS 313068_7, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P18 AS 313068_8, MODEL [C]LINE DETAILLED_BALSHT.313068 USING P20 AS 313068_9, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P2 AS 320095, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P4 AS 320095_1, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P6 AS 320095_2, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P8 AS 320095_3, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P10 AS 320095_4, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P12 AS 320095_5, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P14 AS 320095_6, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P16 AS 320095_7, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P18 AS 320095_8, MODEL [C]LINE DETAILLED_BALSHT.320095 USING P20 AS 320095_9, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P2 AS 320100, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P4 AS 320100_1, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P6 AS 320100_2, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P8 AS 320100_3, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P10 AS 320100_4, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P12 AS 320100_5, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P14 AS 320100_6, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P16 AS 320100_7, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P18 AS 320100_8, MODEL [C]LINE DETAILLED_BALSHT.320100 USING P20 AS 320100_9, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P2 AS 320105, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P4 AS 320105_1, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P6 AS 320105_2, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P8 AS 320105_3, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P10 AS 320105_4, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P12 AS 320105_5, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P14 AS 320105_6, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P16 AS 320105_7, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P18 AS 320105_8, MODEL [C]LINE DETAILLED_BALSHT.320105 USING P20 AS 320105_9, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P2 AS 320110, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P4 AS 320110_1, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P6 AS 320110_2, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P8 AS 320110_3, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P10 AS 320110_4, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P12 AS 320110_5, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P14 AS 320110_6, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P16 AS 320110_7, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P18 AS 320110_8, MODEL [C]LINE DETAILLED_BALSHT.320110 USING P20 AS 320110_9, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P2 AS 320115, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P4 AS 320115_1, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P6 AS 320115_2, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P8 AS 320115_3, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P10 AS 320115_4, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P12 AS 320115_5, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P14 AS 320115_6, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P16 AS 320115_7, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P18 AS 320115_8, MODEL [C]LINE DETAILLED_BALSHT.320115 USING P20 AS 320115_9, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P2 AS 320120, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P4 AS 320120_1, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P6 AS 320120_2, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P8 AS 320120_3, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P10 AS 320120_4, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P12 AS 320120_5, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P14 AS 320120_6, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P16 AS 320120_7, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P18 AS 320120_8, MODEL [C]LINE DETAILLED_BALSHT.320120 USING P20 AS 320120_9, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P2 AS 320125, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P4 AS 320125_1, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P6 AS 320125_2, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P8 AS 320125_3, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P10 AS 320125_4, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P12 AS 320125_5, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P14 AS 320125_6, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P16 AS 320125_7, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P18 AS 320125_8, MODEL [C]LINE DETAILLED_BALSHT.320125 USING P20 AS 320125_9, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P2 AS 320130, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P4 AS 320130_1, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P6 AS 320130_2, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P8 AS 320130_3, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P10 AS 320130_4, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P12 AS 320130_5, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P14 AS 320130_6, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P16 AS 320130_7, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P18 AS 320130_8, MODEL [C]LINE DETAILLED_BALSHT.320130 USING P20 AS 320130_9, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P2 AS 320135, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P4 AS 320135_1, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P6 AS 320135_2, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P8 AS 320135_3, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P10 AS 320135_4, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P12 AS 320135_5, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P14 AS 320135_6, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P16 AS 320135_7, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P18 AS 320135_8, MODEL [C]LINE DETAILLED_BALSHT.320135 USING P20 AS 320135_9, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P2 AS 320140, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P4 AS 320140_1, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P6 AS 320140_2, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P8 AS 320140_3, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P10 AS 320140_4, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P12 AS 320140_5, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P14 AS 320140_6, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P16 AS 320140_7, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P18 AS 320140_8, MODEL [C]LINE DETAILLED_BALSHT.320140 USING P20 AS 320140_9, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P2 AS 320145, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P4 AS 320145_1, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P6 AS 320145_2, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P8 AS 320145_3, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P10 AS 320145_4, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P12 AS 320145_5, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P14 AS 320145_6, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P16 AS 320145_7, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P18 AS 320145_8, MODEL [C]LINE DETAILLED_BALSHT.320145 USING P20 AS 320145_9, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P2 AS 320150, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P4 AS 320150_1, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P6 AS 320150_2, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P8 AS 320150_3, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P10 AS 320150_4, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P12 AS 320150_5, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P14 AS 320150_6, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P16 AS 320150_7, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P18 AS 320150_8, MODEL [C]LINE DETAILLED_BALSHT.320150 USING P20 AS 320150_9, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P2 AS 320155, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P4 AS 320155_1, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P6 AS 320155_2, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P8 AS 320155_3, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P10 AS 320155_4, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P12 AS 320155_5, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P14 AS 320155_6, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P16 AS 320155_7, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P18 AS 320155_8, MODEL [C]LINE DETAILLED_BALSHT.320155 USING P20 AS 320155_9, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P2 AS 320160, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P4 AS 320160_1, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P6 AS 320160_2, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P8 AS 320160_3, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P10 AS 320160_4, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P12 AS 320160_5, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P14 AS 320160_6, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P16 AS 320160_7, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P18 AS 320160_8, MODEL [C]LINE DETAILLED_BALSHT.320160 USING P20 AS 320160_9, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P2 AS 320165, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P4 AS 320165_1, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P6 AS 320165_2, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P8 AS 320165_3, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P10 AS 320165_4, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P12 AS 320165_5, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P14 AS 320165_6, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P16 AS 320165_7, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P18 AS 320165_8, MODEL [C]LINE DETAILLED_BALSHT.320165 USING P20 AS 320165_9, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P2 AS 320170, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P4 AS 320170_1, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P6 AS 320170_2, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P8 AS 320170_3, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P10 AS 320170_4, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P12 AS 320170_5, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P14 AS 320170_6, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P16 AS 320170_7, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P18 AS 320170_8, MODEL [C]LINE DETAILLED_BALSHT.320170 USING P20 AS 320170_9, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P2 AS 320175, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P4 AS 320175_1, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P6 AS 320175_2, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P8 AS 320175_3, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P10 AS 320175_4, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P12 AS 320175_5, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P14 AS 320175_6, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P16 AS 320175_7, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P18 AS 320175_8, MODEL [C]LINE DETAILLED_BALSHT.320175 USING P20 AS 320175_9, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P2 AS 320180, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P4 AS 320180_1, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P6 AS 320180_2, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P8 AS 320180_3, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P10 AS 320180_4, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P12 AS 320180_5, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P14 AS 320180_6, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P16 AS 320180_7, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P18 AS 320180_8, MODEL [C]LINE DETAILLED_BALSHT.320180 USING P20 AS 320180_9, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P2 AS 320184, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P4 AS 320184_1, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P6 AS 320184_2, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P8 AS 320184_3, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P10 AS 320184_4, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P12 AS 320184_5, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P14 AS 320184_6, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P16 AS 320184_7, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P18 AS 320184_8, MODEL [C]LINE DETAILLED_BALSHT.320184 USING P20 AS 320184_9, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P2 AS 320185, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P4 AS 320185_1, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P6 AS 320185_2, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P8 AS 320185_3, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P10 AS 320185_4, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P12 AS 320185_5, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P14 AS 320185_6, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P16 AS 320185_7, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P18 AS 320185_8, MODEL [C]LINE DETAILLED_BALSHT.320185 USING P20 AS 320185_9, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P2 AS 313070, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P4 AS 313070_1, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P6 AS 313070_2, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P8 AS 313070_3, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P10 AS 313070_4, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P12 AS 313070_5, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P14 AS 313070_6, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P16 AS 313070_7, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P18 AS 313070_8, MODEL [C]LINE DETAILLED_BALSHT.313070 USING P20 AS 313070_9, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P2 AS 313071, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P4 AS 313071_1, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P6 AS 313071_2, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P8 AS 313071_3, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P10 AS 313071_4, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P12 AS 313071_5, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P14 AS 313071_6, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P16 AS 313071_7, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P18 AS 313071_8, MODEL [C]LINE DETAILLED_BALSHT.313071 USING P20 AS 313071_9, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P2 AS 320200, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P4 AS 320200_1, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P6 AS 320200_2, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P8 AS 320200_3, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P10 AS 320200_4, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P12 AS 320200_5, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P14 AS 320200_6, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P16 AS 320200_7, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P18 AS 320200_8, MODEL [C]LINE DETAILLED_BALSHT.320200 USING P20 AS 320200_9, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P2 AS 320205, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P4 AS 320205_1, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P6 AS 320205_2, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P8 AS 320205_3, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P10 AS 320205_4, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P12 AS 320205_5, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P14 AS 320205_6, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P16 AS 320205_7, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P18 AS 320205_8, MODEL [C]LINE DETAILLED_BALSHT.320205 USING P20 AS 320205_9, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P2 AS 320210, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P4 AS 320210_1, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P6 AS 320210_2, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P8 AS 320210_3, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P10 AS 320210_4, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P12 AS 320210_5, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P14 AS 320210_6, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P16 AS 320210_7, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P18 AS 320210_8, MODEL [C]LINE DETAILLED_BALSHT.320210 USING P20 AS 320210_9, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P2 AS 320215, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P4 AS 320215_1, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P6 AS 320215_2, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P8 AS 320215_3, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P10 AS 320215_4, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P12 AS 320215_5, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P14 AS 320215_6, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P16 AS 320215_7, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P18 AS 320215_8, MODEL [C]LINE DETAILLED_BALSHT.320215 USING P20 AS 320215_9, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P2 AS 320220, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P4 AS 320220_1, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P6 AS 320220_2, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P8 AS 320220_3, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P10 AS 320220_4, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P12 AS 320220_5, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P14 AS 320220_6, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P16 AS 320220_7, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P18 AS 320220_8, MODEL [C]LINE DETAILLED_BALSHT.320220 USING P20 AS 320220_9, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P2 AS 320225, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P4 AS 320225_1, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P6 AS 320225_2, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P8 AS 320225_3, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P10 AS 320225_4, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P12 AS 320225_5, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P14 AS 320225_6, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P16 AS 320225_7, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P18 AS 320225_8, MODEL [C]LINE DETAILLED_BALSHT.320225 USING P20 AS 320225_9, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P2 AS 320230, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P4 AS 320230_1, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P6 AS 320230_2, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P8 AS 320230_3, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P10 AS 320230_4, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P12 AS 320230_5, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P14 AS 320230_6, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P16 AS 320230_7, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P18 AS 320230_8, MODEL [C]LINE DETAILLED_BALSHT.320230 USING P20 AS 320230_9, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P2 AS 313077, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P4 AS 313077_1, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P6 AS 313077_2, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P8 AS 313077_3, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P10 AS 313077_4, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P12 AS 313077_5, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P14 AS 313077_6, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P16 AS 313077_7, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P18 AS 313077_8, MODEL [C]LINE DETAILLED_BALSHT.313077 USING P20 AS 313077_9, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P2 AS 314011, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P4 AS 314011_1, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P6 AS 314011_2, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P8 AS 314011_3, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P10 AS 314011_4, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P12 AS 314011_5, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P14 AS 314011_6, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P16 AS 314011_7, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P18 AS 314011_8, MODEL [C]LINE DETAILLED_BALSHT.314011 USING P20 AS 314011_9, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P2 AS 321010, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P4 AS 321010_1, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P6 AS 321010_2, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P8 AS 321010_3, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P10 AS 321010_4, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P12 AS 321010_5, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P14 AS 321010_6, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P16 AS 321010_7, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P18 AS 321010_8, MODEL [C]LINE DETAILLED_BALSHT.321010 USING P20 AS 321010_9, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P2 AS 314004, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P4 AS 314004_1, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P6 AS 314004_2, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P8 AS 314004_3, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P10 AS 314004_4, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P12 AS 314004_5, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P14 AS 314004_6, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P16 AS 314004_7, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P18 AS 314004_8, MODEL [C]LINE DETAILLED_BALSHT.314004 USING P20 AS 314004_9, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P2 AS 314046, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P4 AS 314046_1, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P6 AS 314046_2, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P8 AS 314046_3, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P10 AS 314046_4, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P12 AS 314046_5, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P14 AS 314046_6, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P16 AS 314046_7, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P18 AS 314046_8, MODEL [C]LINE DETAILLED_BALSHT.314046 USING P20 AS 314046_9, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P2 AS 321020, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P4 AS 321020_1, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P6 AS 321020_2, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P8 AS 321020_3, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P10 AS 321020_4, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P12 AS 321020_5, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P14 AS 321020_6, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P16 AS 321020_7, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P18 AS 321020_8, MODEL [C]LINE DETAILLED_BALSHT.321020 USING P20 AS 321020_9, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P2 AS 321025, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P4 AS 321025_1, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P6 AS 321025_2, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P8 AS 321025_3, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P10 AS 321025_4, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P12 AS 321025_5, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P14 AS 321025_6, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P16 AS 321025_7, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P18 AS 321025_8, MODEL [C]LINE DETAILLED_BALSHT.321025 USING P20 AS 321025_9, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P2 AS 321030, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P4 AS 321030_1, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P6 AS 321030_2, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P8 AS 321030_3, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P10 AS 321030_4, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P12 AS 321030_5, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P14 AS 321030_6, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P16 AS 321030_7, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P18 AS 321030_8, MODEL [C]LINE DETAILLED_BALSHT.321030 USING P20 AS 321030_9, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P2 AS 321035, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P4 AS 321035_1, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P6 AS 321035_2, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P8 AS 321035_3, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P10 AS 321035_4, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P12 AS 321035_5, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P14 AS 321035_6, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P16 AS 321035_7, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P18 AS 321035_8, MODEL [C]LINE DETAILLED_BALSHT.321035 USING P20 AS 321035_9, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P2 AS 321040, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P4 AS 321040_1, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P6 AS 321040_2, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P8 AS 321040_3, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P10 AS 321040_4, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P12 AS 321040_5, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P14 AS 321040_6, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P16 AS 321040_7, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P18 AS 321040_8, MODEL [C]LINE DETAILLED_BALSHT.321040 USING P20 AS 321040_9, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P2 AS 321045, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P4 AS 321045_1, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P6 AS 321045_2, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P8 AS 321045_3, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P10 AS 321045_4, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P12 AS 321045_5, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P14 AS 321045_6, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P16 AS 321045_7, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P18 AS 321045_8, MODEL [C]LINE DETAILLED_BALSHT.321045 USING P20 AS 321045_9, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P2 AS 321050, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P4 AS 321050_1, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P6 AS 321050_2, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P8 AS 321050_3, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P10 AS 321050_4, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P12 AS 321050_5, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P14 AS 321050_6, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P16 AS 321050_7, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P18 AS 321050_8, MODEL [C]LINE DETAILLED_BALSHT.321050 USING P20 AS 321050_9, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P2 AS 321055, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P4 AS 321055_1, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P6 AS 321055_2, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P8 AS 321055_3, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P10 AS 321055_4, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P12 AS 321055_5, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P14 AS 321055_6, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P16 AS 321055_7, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P18 AS 321055_8, MODEL [C]LINE DETAILLED_BALSHT.321055 USING P20 AS 321055_9, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P2 AS 321060, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P4 AS 321060_1, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P6 AS 321060_2, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P8 AS 321060_3, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P10 AS 321060_4, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P12 AS 321060_5, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P14 AS 321060_6, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P16 AS 321060_7, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P18 AS 321060_8, MODEL [C]LINE DETAILLED_BALSHT.321060 USING P20 AS 321060_9, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P2 AS 314016, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P4 AS 314016_1, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P6 AS 314016_2, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P8 AS 314016_3, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P10 AS 314016_4, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P12 AS 314016_5, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P14 AS 314016_6, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P16 AS 314016_7, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P18 AS 314016_8, MODEL [C]LINE DETAILLED_BALSHT.314016 USING P20 AS 314016_9, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P2 AS 321070, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P4 AS 321070_1, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P6 AS 321070_2, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P8 AS 321070_3, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P10 AS 321070_4, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P12 AS 321070_5, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P14 AS 321070_6, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P16 AS 321070_7, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P18 AS 321070_8, MODEL [C]LINE DETAILLED_BALSHT.321070 USING P20 AS 321070_9, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P2 AS 321075, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P4 AS 321075_1, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P6 AS 321075_2, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P8 AS 321075_3, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P10 AS 321075_4, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P12 AS 321075_5, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P14 AS 321075_6, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P16 AS 321075_7, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P18 AS 321075_8, MODEL [C]LINE DETAILLED_BALSHT.321075 USING P20 AS 321075_9, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P2 AS 314014, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P4 AS 314014_1, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P6 AS 314014_2, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P8 AS 314014_3, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P10 AS 314014_4, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P12 AS 314014_5, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P14 AS 314014_6, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P16 AS 314014_7, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P18 AS 314014_8, MODEL [C]LINE DETAILLED_BALSHT.314014 USING P20 AS 314014_9, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P2 AS 321085, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P4 AS 321085_1, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P6 AS 321085_2, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P8 AS 321085_3, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P10 AS 321085_4, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P12 AS 321085_5, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P14 AS 321085_6, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P16 AS 321085_7, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P18 AS 321085_8, MODEL [C]LINE DETAILLED_BALSHT.321085 USING P20 AS 321085_9, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P2 AS 321090, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P4 AS 321090_1, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P6 AS 321090_2, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P8 AS 321090_3, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P10 AS 321090_4, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P12 AS 321090_5, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P14 AS 321090_6, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P16 AS 321090_7, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P18 AS 321090_8, MODEL [C]LINE DETAILLED_BALSHT.321090 USING P20 AS 321090_9, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P2 AS 321095, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P4 AS 321095_1, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P6 AS 321095_2, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P8 AS 321095_3, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P10 AS 321095_4, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P12 AS 321095_5, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P14 AS 321095_6, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P16 AS 321095_7, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P18 AS 321095_8, MODEL [C]LINE DETAILLED_BALSHT.321095 USING P20 AS 321095_9, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P2 AS 321100, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P4 AS 321100_1, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P6 AS 321100_2, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P8 AS 321100_3, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P10 AS 321100_4, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P12 AS 321100_5, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P14 AS 321100_6, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P16 AS 321100_7, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P18 AS 321100_8, MODEL [C]LINE DETAILLED_BALSHT.321100 USING P20 AS 321100_9, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P2 AS 321105, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P4 AS 321105_1, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P6 AS 321105_2, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P8 AS 321105_3, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P10 AS 321105_4, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P12 AS 321105_5, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P14 AS 321105_6, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P16 AS 321105_7, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P18 AS 321105_8, MODEL [C]LINE DETAILLED_BALSHT.321105 USING P20 AS 321105_9, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P2 AS 321110, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P4 AS 321110_1, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P6 AS 321110_2, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P8 AS 321110_3, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P10 AS 321110_4, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P12 AS 321110_5, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P14 AS 321110_6, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P16 AS 321110_7, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P18 AS 321110_8, MODEL [C]LINE DETAILLED_BALSHT.321110 USING P20 AS 321110_9, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P2 AS 321115, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P4 AS 321115_1, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P6 AS 321115_2, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P8 AS 321115_3, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P10 AS 321115_4, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P12 AS 321115_5, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P14 AS 321115_6, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P16 AS 321115_7, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P18 AS 321115_8, MODEL [C]LINE DETAILLED_BALSHT.321115 USING P20 AS 321115_9, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P2 AS 314018, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P4 AS 314018_1, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P6 AS 314018_2, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P8 AS 314018_3, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P10 AS 314018_4, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P12 AS 314018_5, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P14 AS 314018_6, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P16 AS 314018_7, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P18 AS 314018_8, MODEL [C]LINE DETAILLED_BALSHT.314018 USING P20 AS 314018_9, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P2 AS 314022, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P4 AS 314022_1, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P6 AS 314022_2, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P8 AS 314022_3, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P10 AS 314022_4, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P12 AS 314022_5, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P14 AS 314022_6, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P16 AS 314022_7, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P18 AS 314022_8, MODEL [C]LINE DETAILLED_BALSHT.314022 USING P20 AS 314022_9, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P2 AS 314041, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P4 AS 314041_1, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P6 AS 314041_2, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P8 AS 314041_3, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P10 AS 314041_4, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P12 AS 314041_5, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P14 AS 314041_6, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P16 AS 314041_7, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P18 AS 314041_8, MODEL [C]LINE DETAILLED_BALSHT.314041 USING P20 AS 314041_9, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P2 AS 321135, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P4 AS 321135_1, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P6 AS 321135_2, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P8 AS 321135_3, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P10 AS 321135_4, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P12 AS 321135_5, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P14 AS 321135_6, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P16 AS 321135_7, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P18 AS 321135_8, MODEL [C]LINE DETAILLED_BALSHT.321135 USING P20 AS 321135_9, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P2 AS 314027, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P4 AS 314027_1, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P6 AS 314027_2, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P8 AS 314027_3, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P10 AS 314027_4, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P12 AS 314027_5, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P14 AS 314027_6, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P16 AS 314027_7, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P18 AS 314027_8, MODEL [C]LINE DETAILLED_BALSHT.314027 USING P20 AS 314027_9, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P2 AS 314029, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P4 AS 314029_1, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P6 AS 314029_2, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P8 AS 314029_3, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P10 AS 314029_4, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P12 AS 314029_5, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P14 AS 314029_6, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P16 AS 314029_7, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P18 AS 314029_8, MODEL [C]LINE DETAILLED_BALSHT.314029 USING P20 AS 314029_9, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P2 AS 314026, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P4 AS 314026_1, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P6 AS 314026_2, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P8 AS 314026_3, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P10 AS 314026_4, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P12 AS 314026_5, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P14 AS 314026_6, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P16 AS 314026_7, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P18 AS 314026_8, MODEL [C]LINE DETAILLED_BALSHT.314026 USING P20 AS 314026_9, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P2 AS 314025, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P4 AS 314025_1, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P6 AS 314025_2, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P8 AS 314025_3, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P10 AS 314025_4, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P12 AS 314025_5, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P14 AS 314025_6, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P16 AS 314025_7, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P18 AS 314025_8, MODEL [C]LINE DETAILLED_BALSHT.314025 USING P20 AS 314025_9, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P2 AS 321160, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P4 AS 321160_1, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P6 AS 321160_2, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P8 AS 321160_3, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P10 AS 321160_4, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P12 AS 321160_5, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P14 AS 321160_6, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P16 AS 321160_7, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P18 AS 321160_8, MODEL [C]LINE DETAILLED_BALSHT.321160 USING P20 AS 321160_9, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P2 AS 314032, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P4 AS 314032_1, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P6 AS 314032_2, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P8 AS 314032_3, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P10 AS 314032_4, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P12 AS 314032_5, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P14 AS 314032_6, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P16 AS 314032_7, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P18 AS 314032_8, MODEL [C]LINE DETAILLED_BALSHT.314032 USING P20 AS 314032_9, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P2 AS 314030, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P4 AS 314030_1, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P6 AS 314030_2, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P8 AS 314030_3, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P10 AS 314030_4, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P12 AS 314030_5, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P14 AS 314030_6, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P16 AS 314030_7, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P18 AS 314030_8, MODEL [C]LINE DETAILLED_BALSHT.314030 USING P20 AS 314030_9, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P2 AS 321175, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P4 AS 321175_1, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P6 AS 321175_2, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P8 AS 321175_3, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P10 AS 321175_4, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P12 AS 321175_5, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P14 AS 321175_6, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P16 AS 321175_7, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P18 AS 321175_8, MODEL [C]LINE DETAILLED_BALSHT.321175 USING P20 AS 321175_9, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P2 AS 314036, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P4 AS 314036_1, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P6 AS 314036_2, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P8 AS 314036_3, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P10 AS 314036_4, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P12 AS 314036_5, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P14 AS 314036_6, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P16 AS 314036_7, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P18 AS 314036_8, MODEL [C]LINE DETAILLED_BALSHT.314036 USING P20 AS 314036_9, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P2 AS 321185, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P4 AS 321185_1, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P6 AS 321185_2, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P8 AS 321185_3, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P10 AS 321185_4, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P12 AS 321185_5, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P14 AS 321185_6, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P16 AS 321185_7, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P18 AS 321185_8, MODEL [C]LINE DETAILLED_BALSHT.321185 USING P20 AS 321185_9, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P2 AS 314042, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P4 AS 314042_1, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P6 AS 314042_2, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P8 AS 314042_3, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P10 AS 314042_4, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P12 AS 314042_5, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P14 AS 314042_6, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P16 AS 314042_7, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P18 AS 314042_8, MODEL [C]LINE DETAILLED_BALSHT.314042 USING P20 AS 314042_9, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P2 AS 321210, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P4 AS 321210_1, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P6 AS 321210_2, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P8 AS 321210_3, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P10 AS 321210_4, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P12 AS 321210_5, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P14 AS 321210_6, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P16 AS 321210_7, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P18 AS 321210_8, MODEL [C]LINE DETAILLED_BALSHT.321210 USING P20 AS 321210_9, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P2 AS 321215, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P4 AS 321215_1, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P6 AS 321215_2, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P8 AS 321215_3, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P10 AS 321215_4, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P12 AS 321215_5, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P14 AS 321215_6, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P16 AS 321215_7, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P18 AS 321215_8, MODEL [C]LINE DETAILLED_BALSHT.321215 USING P20 AS 321215_9, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P2 AS 321220, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P4 AS 321220_1, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P6 AS 321220_2, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P8 AS 321220_3, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P10 AS 321220_4, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P12 AS 321220_5, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P14 AS 321220_6, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P16 AS 321220_7, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P18 AS 321220_8, MODEL [C]LINE DETAILLED_BALSHT.321220 USING P20 AS 321220_9, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P21 AS 323000, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P22 AS 323000_1, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P23 AS 323000_2, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P24 AS 323000_3, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P25 AS 323000_4, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P26 AS 323000_5, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P27 AS 323000_6, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P28 AS 323000_7, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P29 AS 323000_8, MODEL [C]LINE DETAILLED_BALSHT.323000 USING P30 AS 323000_9, MODEL [C]LINE DETAILLED_PL.313004 USING P2 AS 313004, MODEL [C]LINE DETAILLED_PL.313004 USING P4 AS 313004_1, MODEL [C]LINE DETAILLED_PL.313004 USING P6 AS 313004_2, MODEL [C]LINE DETAILLED_PL.313004 USING P8 AS 313004_3, MODEL [C]LINE DETAILLED_PL.313004 USING P10 AS 313004_4, MODEL [C]LINE DETAILLED_PL.313004 USING P12 AS 313004_5, MODEL [C]LINE DETAILLED_PL.313004 USING P14 AS 313004_6, MODEL [C]LINE DETAILLED_PL.313004 USING P16 AS 313004_7, MODEL [C]LINE DETAILLED_PL.313004 USING P18 AS 313004_8, MODEL [C]LINE DETAILLED_PL.313004 USING P20 AS 313004_9, MODEL [C]LINE DETAILLED_PL.313000 USING P2 AS 313000, MODEL [C]LINE DETAILLED_PL.313000 USING P4 AS 313000_1, MODEL [C]LINE DETAILLED_PL.313000 USING P6 AS 313000_2, MODEL [C]LINE DETAILLED_PL.313000 USING P8 AS 313000_3, MODEL [C]LINE DETAILLED_PL.313000 USING P10 AS 313000_4, MODEL [C]LINE DETAILLED_PL.313000 USING P12 AS 313000_5, MODEL [C]LINE DETAILLED_PL.313000 USING P14 AS 313000_6, MODEL [C]LINE DETAILLED_PL.313000 USING P16 AS 313000_7, MODEL [C]LINE DETAILLED_PL.313000 USING P18 AS 313000_8, MODEL [C]LINE DETAILLED_PL.313000 USING P20 AS 313000_9, MODEL [C]LINE DETAILLED_PL.313001 USING P2 AS 313001, MODEL [C]LINE DETAILLED_PL.313001 USING P4 AS 313001_1, MODEL [C]LINE DETAILLED_PL.313001 USING P6 AS 313001_2, MODEL [C]LINE DETAILLED_PL.313001 USING P8 AS 313001_3, MODEL [C]LINE DETAILLED_PL.313001 USING P10 AS 313001_4, MODEL [C]LINE DETAILLED_PL.313001 USING P12 AS 313001_5, MODEL [C]LINE DETAILLED_PL.313001 USING P14 AS 313001_6, MODEL [C]LINE DETAILLED_PL.313001 USING P16 AS 313001_7, MODEL [C]LINE DETAILLED_PL.313001 USING P18 AS 313001_8, MODEL [C]LINE DETAILLED_PL.313001 USING P20 AS 313001_9, MODEL [C]LINE DETAILLED_PL.313002 USING P2 AS 313002, MODEL [C]LINE DETAILLED_PL.313002 USING P4 AS 313002_1, MODEL [C]LINE DETAILLED_PL.313002 USING P6 AS 313002_2, MODEL [C]LINE DETAILLED_PL.313002 USING P8 AS 313002_3, MODEL [C]LINE DETAILLED_PL.313002 USING P10 AS 313002_4, MODEL [C]LINE DETAILLED_PL.313002 USING P12 AS 313002_5, MODEL [C]LINE DETAILLED_PL.313002 USING P14 AS 313002_6, MODEL [C]LINE DETAILLED_PL.313002 USING P16 AS 313002_7, MODEL [C]LINE DETAILLED_PL.313002 USING P18 AS 313002_8, MODEL [C]LINE DETAILLED_PL.313002 USING P20 AS 313002_9, MODEL [C]LINE DETAILLED_PL.313003 USING P2 AS 313003, MODEL [C]LINE DETAILLED_PL.313003 USING P4 AS 313003_1, MODEL [C]LINE DETAILLED_PL.313003 USING P6 AS 313003_2, MODEL [C]LINE DETAILLED_PL.313003 USING P8 AS 313003_3, MODEL [C]LINE DETAILLED_PL.313003 USING P10 AS 313003_4, MODEL [C]LINE DETAILLED_PL.313003 USING P12 AS 313003_5, MODEL [C]LINE DETAILLED_PL.313003 USING P14 AS 313003_6, MODEL [C]LINE DETAILLED_PL.313003 USING P16 AS 313003_7, MODEL [C]LINE DETAILLED_PL.313003 USING P18 AS 313003_8, MODEL [C]LINE DETAILLED_PL.313003 USING P20 AS 313003_9, MODEL [C]LINE DETAILLED_PL.322199 USING P2 AS 322199, MODEL [C]LINE DETAILLED_PL.322199 USING P4 AS 322199_1, MODEL [C]LINE DETAILLED_PL.322199 USING P6 AS 322199_2, MODEL [C]LINE DETAILLED_PL.322199 USING P8 AS 322199_3, MODEL [C]LINE DETAILLED_PL.322199 USING P10 AS 322199_4, MODEL [C]LINE DETAILLED_PL.322199 USING P12 AS 322199_5, MODEL [C]LINE DETAILLED_PL.322199 USING P14 AS 322199_6, MODEL [C]LINE DETAILLED_PL.322199 USING P16 AS 322199_7, MODEL [C]LINE DETAILLED_PL.322199 USING P18 AS 322199_8, MODEL [C]LINE DETAILLED_PL.322199 USING P20 AS 322199_9, MODEL [C]LINE DETAILLED_PL.322020 USING P2 AS 322020, MODEL [C]LINE DETAILLED_PL.322020 USING P4 AS 322020_1, MODEL [C]LINE DETAILLED_PL.322020 USING P6 AS 322020_2, MODEL [C]LINE DETAILLED_PL.322020 USING P8 AS 322020_3, MODEL [C]LINE DETAILLED_PL.322020 USING P10 AS 322020_4, MODEL [C]LINE DETAILLED_PL.322020 USING P12 AS 322020_5, MODEL [C]LINE DETAILLED_PL.322020 USING P14 AS 322020_6, MODEL [C]LINE DETAILLED_PL.322020 USING P16 AS 322020_7, MODEL [C]LINE DETAILLED_PL.322020 USING P18 AS 322020_8, MODEL [C]LINE DETAILLED_PL.322020 USING P20 AS 322020_9, MODEL [C]LINE DETAILLED_PL.322035 USING P2 AS 322035, MODEL [C]LINE DETAILLED_PL.322035 USING P4 AS 322035_1, MODEL [C]LINE DETAILLED_PL.322035 USING P6 AS 322035_2, MODEL [C]LINE DETAILLED_PL.322035 USING P8 AS 322035_3, MODEL [C]LINE DETAILLED_PL.322035 USING P10 AS 322035_4, MODEL [C]LINE DETAILLED_PL.322035 USING P12 AS 322035_5, MODEL [C]LINE DETAILLED_PL.322035 USING P14 AS 322035_6, MODEL [C]LINE DETAILLED_PL.322035 USING P16 AS 322035_7, MODEL [C]LINE DETAILLED_PL.322035 USING P18 AS 322035_8, MODEL [C]LINE DETAILLED_PL.322035 USING P20 AS 322035_9, MODEL [C]LINE DETAILLED_PL.313018 USING P2 AS 313018, MODEL [C]LINE DETAILLED_PL.313018 USING P4 AS 313018_1, MODEL [C]LINE DETAILLED_PL.313018 USING P6 AS 313018_2, MODEL [C]LINE DETAILLED_PL.313018 USING P8 AS 313018_3, MODEL [C]LINE DETAILLED_PL.313018 USING P10 AS 313018_4, MODEL [C]LINE DETAILLED_PL.313018 USING P12 AS 313018_5, MODEL [C]LINE DETAILLED_PL.313018 USING P14 AS 313018_6, MODEL [C]LINE DETAILLED_PL.313018 USING P16 AS 313018_7, MODEL [C]LINE DETAILLED_PL.313018 USING P18 AS 313018_8, MODEL [C]LINE DETAILLED_PL.313018 USING P20 AS 313018_9, MODEL [C]LINE DETAILLED_PL.313021 USING P2 AS 313021, MODEL [C]LINE DETAILLED_PL.313021 USING P4 AS 313021_1, MODEL [C]LINE DETAILLED_PL.313021 USING P6 AS 313021_2, MODEL [C]LINE DETAILLED_PL.313021 USING P8 AS 313021_3, MODEL [C]LINE DETAILLED_PL.313021 USING P10 AS 313021_4, MODEL [C]LINE DETAILLED_PL.313021 USING P12 AS 313021_5, MODEL [C]LINE DETAILLED_PL.313021 USING P14 AS 313021_6, MODEL [C]LINE DETAILLED_PL.313021 USING P16 AS 313021_7, MODEL [C]LINE DETAILLED_PL.313021 USING P18 AS 313021_8, MODEL [C]LINE DETAILLED_PL.313021 USING P20 AS 313021_9, MODEL [C]LINE DETAILLED_PL.313019 USING P2 AS 313019, MODEL [C]LINE DETAILLED_PL.313019 USING P4 AS 313019_1, MODEL [C]LINE DETAILLED_PL.313019 USING P6 AS 313019_2, MODEL [C]LINE DETAILLED_PL.313019 USING P8 AS 313019_3, MODEL [C]LINE DETAILLED_PL.313019 USING P10 AS 313019_4, MODEL [C]LINE DETAILLED_PL.313019 USING P12 AS 313019_5, MODEL [C]LINE DETAILLED_PL.313019 USING P14 AS 313019_6, MODEL [C]LINE DETAILLED_PL.313019 USING P16 AS 313019_7, MODEL [C]LINE DETAILLED_PL.313019 USING P18 AS 313019_8, MODEL [C]LINE DETAILLED_PL.313019 USING P20 AS 313019_9, MODEL [C]LINE DETAILLED_PL.313020 USING P2 AS 313020, MODEL [C]LINE DETAILLED_PL.313020 USING P4 AS 313020_1, MODEL [C]LINE DETAILLED_PL.313020 USING P6 AS 313020_2, MODEL [C]LINE DETAILLED_PL.313020 USING P8 AS 313020_3, MODEL [C]LINE DETAILLED_PL.313020 USING P10 AS 313020_4, MODEL [C]LINE DETAILLED_PL.313020 USING P12 AS 313020_5, MODEL [C]LINE DETAILLED_PL.313020 USING P14 AS 313020_6, MODEL [C]LINE DETAILLED_PL.313020 USING P16 AS 313020_7, MODEL [C]LINE DETAILLED_PL.313020 USING P18 AS 313020_8, MODEL [C]LINE DETAILLED_PL.313020 USING P20 AS 313020_9, MODEL [C]LINE DETAILLED_PL.313022 USING P2 AS 313022, MODEL [C]LINE DETAILLED_PL.313022 USING P4 AS 313022_1, MODEL [C]LINE DETAILLED_PL.313022 USING P6 AS 313022_2, MODEL [C]LINE DETAILLED_PL.313022 USING P8 AS 313022_3, MODEL [C]LINE DETAILLED_PL.313022 USING P10 AS 313022_4, MODEL [C]LINE DETAILLED_PL.313022 USING P12 AS 313022_5, MODEL [C]LINE DETAILLED_PL.313022 USING P14 AS 313022_6, MODEL [C]LINE DETAILLED_PL.313022 USING P16 AS 313022_7, MODEL [C]LINE DETAILLED_PL.313022 USING P18 AS 313022_8, MODEL [C]LINE DETAILLED_PL.313022 USING P20 AS 313022_9, MODEL [C]LINE DETAILLED_PL.313023 USING P2 AS 313023, MODEL [C]LINE DETAILLED_PL.313023 USING P4 AS 313023_1, MODEL [C]LINE DETAILLED_PL.313023 USING P6 AS 313023_2, MODEL [C]LINE DETAILLED_PL.313023 USING P8 AS 313023_3, MODEL [C]LINE DETAILLED_PL.313023 USING P10 AS 313023_4, MODEL [C]LINE DETAILLED_PL.313023 USING P12 AS 313023_5, MODEL [C]LINE DETAILLED_PL.313023 USING P14 AS 313023_6, MODEL [C]LINE DETAILLED_PL.313023 USING P16 AS 313023_7, MODEL [C]LINE DETAILLED_PL.313023 USING P18 AS 313023_8, MODEL [C]LINE DETAILLED_PL.313023 USING P20 AS 313023_9, MODEL [C]LINE DETAILLED_PL.313024 USING P2 AS 313024, MODEL [C]LINE DETAILLED_PL.313024 USING P4 AS 313024_1, MODEL [C]LINE DETAILLED_PL.313024 USING P6 AS 313024_2, MODEL [C]LINE DETAILLED_PL.313024 USING P8 AS 313024_3, MODEL [C]LINE DETAILLED_PL.313024 USING P10 AS 313024_4, MODEL [C]LINE DETAILLED_PL.313024 USING P12 AS 313024_5, MODEL [C]LINE DETAILLED_PL.313024 USING P14 AS 313024_6, MODEL [C]LINE DETAILLED_PL.313024 USING P16 AS 313024_7, MODEL [C]LINE DETAILLED_PL.313024 USING P18 AS 313024_8, MODEL [C]LINE DETAILLED_PL.313024 USING P20 AS 313024_9, MODEL [C]LINE DETAILLED_PL.313025 USING P2 AS 313025, MODEL [C]LINE DETAILLED_PL.313025 USING P4 AS 313025_1, MODEL [C]LINE DETAILLED_PL.313025 USING P6 AS 313025_2, MODEL [C]LINE DETAILLED_PL.313025 USING P8 AS 313025_3, MODEL [C]LINE DETAILLED_PL.313025 USING P10 AS 313025_4, MODEL [C]LINE DETAILLED_PL.313025 USING P12 AS 313025_5, MODEL [C]LINE DETAILLED_PL.313025 USING P14 AS 313025_6, MODEL [C]LINE DETAILLED_PL.313025 USING P16 AS 313025_7, MODEL [C]LINE DETAILLED_PL.313025 USING P18 AS 313025_8, MODEL [C]LINE DETAILLED_PL.313025 USING P20 AS 313025_9, MODEL [C]LINE DETAILLED_PL.313026 USING P2 AS 313026, MODEL [C]LINE DETAILLED_PL.313026 USING P4 AS 313026_1, MODEL [C]LINE DETAILLED_PL.313026 USING P6 AS 313026_2, MODEL [C]LINE DETAILLED_PL.313026 USING P8 AS 313026_3, MODEL [C]LINE DETAILLED_PL.313026 USING P10 AS 313026_4, MODEL [C]LINE DETAILLED_PL.313026 USING P12 AS 313026_5, MODEL [C]LINE DETAILLED_PL.313026 USING P14 AS 313026_6, MODEL [C]LINE DETAILLED_PL.313026 USING P16 AS 313026_7, MODEL [C]LINE DETAILLED_PL.313026 USING P18 AS 313026_8, MODEL [C]LINE DETAILLED_PL.313026 USING P20 AS 313026_9, MODEL [C]LINE DETAILLED_PL.313027 USING P2 AS 313027, MODEL [C]LINE DETAILLED_PL.313027 USING P4 AS 313027_1, MODEL [C]LINE DETAILLED_PL.313027 USING P6 AS 313027_2, MODEL [C]LINE DETAILLED_PL.313027 USING P8 AS 313027_3, MODEL [C]LINE DETAILLED_PL.313027 USING P10 AS 313027_4, MODEL [C]LINE DETAILLED_PL.313027 USING P12 AS 313027_5, MODEL [C]LINE DETAILLED_PL.313027 USING P14 AS 313027_6, MODEL [C]LINE DETAILLED_PL.313027 USING P16 AS 313027_7, MODEL [C]LINE DETAILLED_PL.313027 USING P18 AS 313027_8, MODEL [C]LINE DETAILLED_PL.313027 USING P20 AS 313027_9, MODEL [C]LINE DETAILLED_PL.322080 USING P2 AS 322080, MODEL [C]LINE DETAILLED_PL.322080 USING P4 AS 322080_1, MODEL [C]LINE DETAILLED_PL.322080 USING P6 AS 322080_2, MODEL [C]LINE DETAILLED_PL.322080 USING P8 AS 322080_3, MODEL [C]LINE DETAILLED_PL.322080 USING P10 AS 322080_4, MODEL [C]LINE DETAILLED_PL.322080 USING P12 AS 322080_5, MODEL [C]LINE DETAILLED_PL.322080 USING P14 AS 322080_6, MODEL [C]LINE DETAILLED_PL.322080 USING P16 AS 322080_7, MODEL [C]LINE DETAILLED_PL.322080 USING P18 AS 322080_8, MODEL [C]LINE DETAILLED_PL.322080 USING P20 AS 322080_9, MODEL [C]LINE DETAILLED_PL.313034 USING P2 AS 313034, MODEL [C]LINE DETAILLED_PL.313034 USING P4 AS 313034_1, MODEL [C]LINE DETAILLED_PL.313034 USING P6 AS 313034_2, MODEL [C]LINE DETAILLED_PL.313034 USING P8 AS 313034_3, MODEL [C]LINE DETAILLED_PL.313034 USING P10 AS 313034_4, MODEL [C]LINE DETAILLED_PL.313034 USING P12 AS 313034_5, MODEL [C]LINE DETAILLED_PL.313034 USING P14 AS 313034_6, MODEL [C]LINE DETAILLED_PL.313034 USING P16 AS 313034_7, MODEL [C]LINE DETAILLED_PL.313034 USING P18 AS 313034_8, MODEL [C]LINE DETAILLED_PL.313034 USING P20 AS 313034_9, MODEL [C]LINE DETAILLED_PL.313035 USING P2 AS 313035, MODEL [C]LINE DETAILLED_PL.313035 USING P4 AS 313035_1, MODEL [C]LINE DETAILLED_PL.313035 USING P6 AS 313035_2, MODEL [C]LINE DETAILLED_PL.313035 USING P8 AS 313035_3, MODEL [C]LINE DETAILLED_PL.313035 USING P10 AS 313035_4, MODEL [C]LINE DETAILLED_PL.313035 USING P12 AS 313035_5, MODEL [C]LINE DETAILLED_PL.313035 USING P14 AS 313035_6, MODEL [C]LINE DETAILLED_PL.313035 USING P16 AS 313035_7, MODEL [C]LINE DETAILLED_PL.313035 USING P18 AS 313035_8, MODEL [C]LINE DETAILLED_PL.313035 USING P20 AS 313035_9, MODEL [C]LINE DETAILLED_PL.313037 USING P2 AS 313037, MODEL [C]LINE DETAILLED_PL.313037 USING P4 AS 313037_1, MODEL [C]LINE DETAILLED_PL.313037 USING P6 AS 313037_2, MODEL [C]LINE DETAILLED_PL.313037 USING P8 AS 313037_3, MODEL [C]LINE DETAILLED_PL.313037 USING P10 AS 313037_4, MODEL [C]LINE DETAILLED_PL.313037 USING P12 AS 313037_5, MODEL [C]LINE DETAILLED_PL.313037 USING P14 AS 313037_6, MODEL [C]LINE DETAILLED_PL.313037 USING P16 AS 313037_7, MODEL [C]LINE DETAILLED_PL.313037 USING P18 AS 313037_8, MODEL [C]LINE DETAILLED_PL.313037 USING P20 AS 313037_9, MODEL [C]LINE DETAILLED_PL.313038 USING P2 AS 313038, MODEL [C]LINE DETAILLED_PL.313038 USING P4 AS 313038_1, MODEL [C]LINE DETAILLED_PL.313038 USING P6 AS 313038_2, MODEL [C]LINE DETAILLED_PL.313038 USING P8 AS 313038_3, MODEL [C]LINE DETAILLED_PL.313038 USING P10 AS 313038_4, MODEL [C]LINE DETAILLED_PL.313038 USING P12 AS 313038_5, MODEL [C]LINE DETAILLED_PL.313038 USING P14 AS 313038_6, MODEL [C]LINE DETAILLED_PL.313038 USING P16 AS 313038_7, MODEL [C]LINE DETAILLED_PL.313038 USING P18 AS 313038_8, MODEL [C]LINE DETAILLED_PL.313038 USING P20 AS 313038_9, MODEL [C]LINE DETAILLED_PL.322115 USING P2 AS 322115, MODEL [C]LINE DETAILLED_PL.322115 USING P4 AS 322115_1, MODEL [C]LINE DETAILLED_PL.322115 USING P6 AS 322115_2, MODEL [C]LINE DETAILLED_PL.322115 USING P8 AS 322115_3, MODEL [C]LINE DETAILLED_PL.322115 USING P10 AS 322115_4, MODEL [C]LINE DETAILLED_PL.322115 USING P12 AS 322115_5, MODEL [C]LINE DETAILLED_PL.322115 USING P14 AS 322115_6, MODEL [C]LINE DETAILLED_PL.322115 USING P16 AS 322115_7, MODEL [C]LINE DETAILLED_PL.322115 USING P18 AS 322115_8, MODEL [C]LINE DETAILLED_PL.322115 USING P20 AS 322115_9, MODEL [C]LINE DETAILLED_PL.313043 USING P2 AS 313043, MODEL [C]LINE DETAILLED_PL.313043 USING P4 AS 313043_1, MODEL [C]LINE DETAILLED_PL.313043 USING P6 AS 313043_2, MODEL [C]LINE DETAILLED_PL.313043 USING P8 AS 313043_3, MODEL [C]LINE DETAILLED_PL.313043 USING P10 AS 313043_4, MODEL [C]LINE DETAILLED_PL.313043 USING P12 AS 313043_5, MODEL [C]LINE DETAILLED_PL.313043 USING P14 AS 313043_6, MODEL [C]LINE DETAILLED_PL.313043 USING P16 AS 313043_7, MODEL [C]LINE DETAILLED_PL.313043 USING P18 AS 313043_8, MODEL [C]LINE DETAILLED_PL.313043 USING P20 AS 313043_9, MODEL [C]LINE DETAILLED_PL.313044 USING P2 AS 313044, MODEL [C]LINE DETAILLED_PL.313044 USING P4 AS 313044_1, MODEL [C]LINE DETAILLED_PL.313044 USING P6 AS 313044_2, MODEL [C]LINE DETAILLED_PL.313044 USING P8 AS 313044_3, MODEL [C]LINE DETAILLED_PL.313044 USING P10 AS 313044_4, MODEL [C]LINE DETAILLED_PL.313044 USING P12 AS 313044_5, MODEL [C]LINE DETAILLED_PL.313044 USING P14 AS 313044_6, MODEL [C]LINE DETAILLED_PL.313044 USING P16 AS 313044_7, MODEL [C]LINE DETAILLED_PL.313044 USING P18 AS 313044_8, MODEL [C]LINE DETAILLED_PL.313044 USING P20 AS 313044_9, MODEL [C]LINE DETAILLED_PL.313045 USING P2 AS 313045, MODEL [C]LINE DETAILLED_PL.313045 USING P4 AS 313045_1, MODEL [C]LINE DETAILLED_PL.313045 USING P6 AS 313045_2, MODEL [C]LINE DETAILLED_PL.313045 USING P8 AS 313045_3, MODEL [C]LINE DETAILLED_PL.313045 USING P10 AS 313045_4, MODEL [C]LINE DETAILLED_PL.313045 USING P12 AS 313045_5, MODEL [C]LINE DETAILLED_PL.313045 USING P14 AS 313045_6, MODEL [C]LINE DETAILLED_PL.313045 USING P16 AS 313045_7, MODEL [C]LINE DETAILLED_PL.313045 USING P18 AS 313045_8, MODEL [C]LINE DETAILLED_PL.313045 USING P20 AS 313045_9, MODEL [C]LINE DETAILLED_PL.313046 USING P2 AS 313046, MODEL [C]LINE DETAILLED_PL.313046 USING P4 AS 313046_1, MODEL [C]LINE DETAILLED_PL.313046 USING P6 AS 313046_2, MODEL [C]LINE DETAILLED_PL.313046 USING P8 AS 313046_3, MODEL [C]LINE DETAILLED_PL.313046 USING P10 AS 313046_4, MODEL [C]LINE DETAILLED_PL.313046 USING P12 AS 313046_5, MODEL [C]LINE DETAILLED_PL.313046 USING P14 AS 313046_6, MODEL [C]LINE DETAILLED_PL.313046 USING P16 AS 313046_7, MODEL [C]LINE DETAILLED_PL.313046 USING P18 AS 313046_8, MODEL [C]LINE DETAILLED_PL.313046 USING P20 AS 313046_9, MODEL [C]LINE DETAILLED_PL.313047 USING P2 AS 313047, MODEL [C]LINE DETAILLED_PL.313047 USING P4 AS 313047_1, MODEL [C]LINE DETAILLED_PL.313047 USING P6 AS 313047_2, MODEL [C]LINE DETAILLED_PL.313047 USING P8 AS 313047_3, MODEL [C]LINE DETAILLED_PL.313047 USING P10 AS 313047_4, MODEL [C]LINE DETAILLED_PL.313047 USING P12 AS 313047_5, MODEL [C]LINE DETAILLED_PL.313047 USING P14 AS 313047_6, MODEL [C]LINE DETAILLED_PL.313047 USING P16 AS 313047_7, MODEL [C]LINE DETAILLED_PL.313047 USING P18 AS 313047_8, MODEL [C]LINE DETAILLED_PL.313047 USING P20 AS 313047_9, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P2 AS 315500, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P4 AS 315500_1, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P6 AS 315500_2, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P8 AS 315500_3, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P10 AS 315500_4, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P12 AS 315500_5, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P14 AS 315500_6, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P16 AS 315500_7, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P18 AS 315500_8, MODEL [CWVB]LINE CASHFLOW_WVB.315500 USING P20 AS 315500_9, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P2 AS 315501, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P4 AS 315501_1, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P6 AS 315501_2, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P8 AS 315501_3, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P10 AS 315501_4, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P12 AS 315501_5, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P14 AS 315501_6, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P16 AS 315501_7, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P18 AS 315501_8, MODEL [CWVB]LINE CASHFLOW_WVB.315501 USING P20 AS 315501_9, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P2 AS 315502, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P4 AS 315502_1, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P6 AS 315502_2, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P8 AS 315502_3, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P10 AS 315502_4, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P12 AS 315502_5, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P14 AS 315502_6, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P16 AS 315502_7, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P18 AS 315502_8, MODEL [CWVB]LINE CASHFLOW_WVB.315502 USING P20 AS 315502_9, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P2 AS 315503, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P4 AS 315503_1, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P6 AS 315503_2, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P8 AS 315503_3, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P10 AS 315503_4, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P12 AS 315503_5, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P14 AS 315503_6, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P16 AS 315503_7, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P18 AS 315503_8, MODEL [CWVB]LINE CASHFLOW_WVB.315503 USING P20 AS 315503_9, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P2 AS 315504, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P4 AS 315504_1, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P6 AS 315504_2, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P8 AS 315504_3, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P10 AS 315504_4, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P12 AS 315504_5, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P14 AS 315504_6, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P16 AS 315504_7, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P18 AS 315504_8, MODEL [CWVB]LINE CASHFLOW_WVB.315504 USING P20 AS 315504_9, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P2 AS 315505, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P4 AS 315505_1, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P6 AS 315505_2, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P8 AS 315505_3, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P10 AS 315505_4, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P12 AS 315505_5, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P14 AS 315505_6, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P16 AS 315505_7, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P18 AS 315505_8, MODEL [CWVB]LINE CASHFLOW_WVB.315505 USING P20 AS 315505_9, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P2 AS 315506, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P4 AS 315506_1, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P6 AS 315506_2, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P8 AS 315506_3, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P10 AS 315506_4, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P12 AS 315506_5, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P14 AS 315506_6, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P16 AS 315506_7, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P18 AS 315506_8, MODEL [CWVB]LINE CASHFLOW_WVB.315506 USING P20 AS 315506_9, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P2 AS 315508, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P4 AS 315508_1, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P6 AS 315508_2, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P8 AS 315508_3, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P10 AS 315508_4, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P12 AS 315508_5, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P14 AS 315508_6, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P16 AS 315508_7, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P18 AS 315508_8, MODEL [CWVB]LINE CASHFLOW_WVB.315508 USING P20 AS 315508_9, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P2 AS 315509, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P4 AS 315509_1, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P6 AS 315509_2, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P8 AS 315509_3, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P10 AS 315509_4, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P12 AS 315509_5, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P14 AS 315509_6, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P16 AS 315509_7, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P18 AS 315509_8, MODEL [CWVB]LINE CASHFLOW_WVB.315509 USING P20 AS 315509_9, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P2 AS 315510, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P4 AS 315510_1, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P6 AS 315510_2, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P8 AS 315510_3, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P10 AS 315510_4, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P12 AS 315510_5, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P14 AS 315510_6, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P16 AS 315510_7, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P18 AS 315510_8, MODEL [CWVB]LINE CASHFLOW_WVB.315510 USING P20 AS 315510_9, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P2 AS 315511, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P4 AS 315511_1, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P6 AS 315511_2, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P8 AS 315511_3, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P10 AS 315511_4, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P12 AS 315511_5, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P14 AS 315511_6, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P16 AS 315511_7, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P18 AS 315511_8, MODEL [CWVB]LINE CASHFLOW_WVB.315511 USING P20 AS 315511_9, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P2 AS 315512, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P4 AS 315512_1, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P6 AS 315512_2, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P8 AS 315512_3, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P10 AS 315512_4, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P12 AS 315512_5, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P14 AS 315512_6, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P16 AS 315512_7, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P18 AS 315512_8, MODEL [CWVB]LINE CASHFLOW_WVB.315512 USING P20 AS 315512_9, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P2 AS 315513, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P4 AS 315513_1, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P6 AS 315513_2, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P8 AS 315513_3, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P10 AS 315513_4, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P12 AS 315513_5, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P14 AS 315513_6, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P16 AS 315513_7, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P18 AS 315513_8, MODEL [CWVB]LINE CASHFLOW_WVB.315513 USING P20 AS 315513_9, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P2 AS 315507, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P4 AS 315507_1, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P6 AS 315507_2, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P8 AS 315507_3, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P10 AS 315507_4, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P12 AS 315507_5, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P14 AS 315507_6, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P16 AS 315507_7, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P18 AS 315507_8, MODEL [CWVB]LINE CASHFLOW_WVB.315507 USING P20 AS 315507_9, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P2 AS 315514, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P4 AS 315514_1, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P6 AS 315514_2, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P8 AS 315514_3, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P10 AS 315514_4, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P12 AS 315514_5, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P14 AS 315514_6, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P16 AS 315514_7, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P18 AS 315514_8, MODEL [CWVB]LINE CASHFLOW_WVB.315514 USING P20 AS 315514_9, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P2 AS 315515, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P4 AS 315515_1, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P6 AS 315515_2, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P8 AS 315515_3, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P10 AS 315515_4, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P12 AS 315515_5, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P14 AS 315515_6, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P16 AS 315515_7, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P18 AS 315515_8, MODEL [CWVB]LINE CASHFLOW_WVB.315515 USING P20 AS 315515_9, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P2 AS 315516, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P4 AS 315516_1, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P6 AS 315516_2, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P8 AS 315516_3, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P10 AS 315516_4, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P12 AS 315516_5, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P14 AS 315516_6, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P16 AS 315516_7, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P18 AS 315516_8, MODEL [CWVB]LINE CASHFLOW_WVB.315516 USING P20 AS 315516_9, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P2 AS 315517, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P4 AS 315517_1, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P6 AS 315517_2, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P8 AS 315517_3, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P10 AS 315517_4, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P12 AS 315517_5, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P14 AS 315517_6, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P16 AS 315517_7, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P18 AS 315517_8, MODEL [CWVB]LINE CASHFLOW_WVB.315517 USING P20 AS 315517_9, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P2 AS 315518, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P4 AS 315518_1, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P6 AS 315518_2, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P8 AS 315518_3, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P10 AS 315518_4, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P12 AS 315518_5, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P14 AS 315518_6, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P16 AS 315518_7, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P18 AS 315518_8, MODEL [CWVB]LINE CASHFLOW_WVB.315518 USING P20 AS 315518_9, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P2 AS 315519, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P4 AS 315519_1, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P6 AS 315519_2, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P8 AS 315519_3, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P10 AS 315519_4, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P12 AS 315519_5, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P14 AS 315519_6, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P16 AS 315519_7, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P18 AS 315519_8, MODEL [CWVB]LINE CASHFLOW_WVB.315519 USING P20 AS 315519_9, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P2 AS 315520, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P4 AS 315520_1, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P6 AS 315520_2, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P8 AS 315520_3, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P10 AS 315520_4, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P12 AS 315520_5, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P14 AS 315520_6, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P16 AS 315520_7, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P18 AS 315520_8, MODEL [CWVB]LINE CASHFLOW_WVB.315520 USING P20 AS 315520_9, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P2 AS 315521, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P4 AS 315521_1, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P6 AS 315521_2, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P8 AS 315521_3, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P10 AS 315521_4, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P12 AS 315521_5, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P14 AS 315521_6, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P16 AS 315521_7, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P18 AS 315521_8, MODEL [CWVB]LINE CASHFLOW_WVB.315521 USING P20 AS 315521_9, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P2 AS 315522, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P4 AS 315522_1, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P6 AS 315522_2, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P8 AS 315522_3, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P10 AS 315522_4, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P12 AS 315522_5, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P14 AS 315522_6, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P16 AS 315522_7, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P18 AS 315522_8, MODEL [CWVB]LINE CASHFLOW_WVB.315522 USING P20 AS 315522_9, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P2 AS 315524, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P4 AS 315524_1, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P6 AS 315524_2, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P8 AS 315524_3, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P10 AS 315524_4, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P12 AS 315524_5, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P14 AS 315524_6, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P16 AS 315524_7, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P18 AS 315524_8, MODEL [CWVB]LINE CASHFLOW_WVB.315524 USING P20 AS 315524_9, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P2 AS 315525, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P4 AS 315525_1, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P6 AS 315525_2, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P8 AS 315525_3, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P10 AS 315525_4, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P12 AS 315525_5, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P14 AS 315525_6, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P16 AS 315525_7, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P18 AS 315525_8, MODEL [CWVB]LINE CASHFLOW_WVB.315525 USING P20 AS 315525_9, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P2 AS 315523, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P4 AS 315523_1, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P6 AS 315523_2, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P8 AS 315523_3, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P10 AS 315523_4, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P12 AS 315523_5, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P14 AS 315523_6, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P16 AS 315523_7, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P18 AS 315523_8, MODEL [CWVB]LINE CASHFLOW_WVB.315523 USING P20 AS 315523_9, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P2 AS 315526, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P4 AS 315526_1, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P6 AS 315526_2, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P8 AS 315526_3, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P10 AS 315526_4, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P12 AS 315526_5, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P14 AS 315526_6, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P16 AS 315526_7, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P18 AS 315526_8, MODEL [CWVB]LINE CASHFLOW_WVB.315526 USING P20 AS 315526_9, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P2 AS 315528, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P4 AS 315528_1, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P6 AS 315528_2, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P8 AS 315528_3, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P10 AS 315528_4, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P12 AS 315528_5, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P14 AS 315528_6, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P16 AS 315528_7, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P18 AS 315528_8, MODEL [CWVB]LINE CASHFLOW_WVB.315528 USING P20 AS 315528_9, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P2 AS 315529, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P4 AS 315529_1, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P6 AS 315529_2, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P8 AS 315529_3, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P10 AS 315529_4, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P12 AS 315529_5, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P14 AS 315529_6, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P16 AS 315529_7, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P18 AS 315529_8, MODEL [CWVB]LINE CASHFLOW_WVB.315529 USING P20 AS 315529_9, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P2 AS 315530, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P4 AS 315530_1, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P6 AS 315530_2, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P8 AS 315530_3, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P10 AS 315530_4, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P12 AS 315530_5, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P14 AS 315530_6, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P16 AS 315530_7, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P18 AS 315530_8, MODEL [CWVB]LINE CASHFLOW_WVB.315530 USING P20 AS 315530_9, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P2 AS 315531, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P4 AS 315531_1, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P6 AS 315531_2, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P8 AS 315531_3, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P10 AS 315531_4, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P12 AS 315531_5, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P14 AS 315531_6, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P16 AS 315531_7, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P18 AS 315531_8, MODEL [CWVB]LINE CASHFLOW_WVB.315531 USING P20 AS 315531_9, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P2 AS 315532, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P4 AS 315532_1, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P6 AS 315532_2, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P8 AS 315532_3, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P10 AS 315532_4, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P12 AS 315532_5, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P14 AS 315532_6, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P16 AS 315532_7, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P18 AS 315532_8, MODEL [CWVB]LINE CASHFLOW_WVB.315532 USING P20 AS 315532_9, MODEL [C]LINE RATIOS.509 USING P21 AS RSHF_10, MODEL [C]LINE RATIOS.509 USING P22 AS RSHF_11, MODEL [C]LINE RATIOS.509 USING P23 AS RSHF_12, MODEL [C]LINE RATIOS.509 USING P24 AS RSHF_13, MODEL [C]LINE RATIOS.509 USING P25 AS RSHF_14, MODEL [C]LINE RATIOS.509 USING P26 AS RSHF_15, MODEL [C]LINE RATIOS.509 USING P27 AS RSHF_16, MODEL [C]LINE RATIOS.509 USING P28 AS RSHF_17, MODEL [C]LINE RATIOS.509 USING P29 AS RSHF_18, MODEL [C]LINE RATIOS.509 USING P30 AS RSHF_19, MODEL [C]LINE RATIOS.510 USING P21 AS RCEM_10, MODEL [C]LINE RATIOS.510 USING P22 AS RCEM_11, MODEL [C]LINE RATIOS.510 USING P23 AS RCEM_12, MODEL [C]LINE RATIOS.510 USING P24 AS RCEM_13, MODEL [C]LINE RATIOS.510 USING P25 AS RCEM_14, MODEL [C]LINE RATIOS.510 USING P26 AS RCEM_15, MODEL [C]LINE RATIOS.510 USING P27 AS RCEM_16, MODEL [C]LINE RATIOS.510 USING P28 AS RCEM_17, MODEL [C]LINE RATIOS.510 USING P29 AS RCEM_18, MODEL [C]LINE RATIOS.510 USING P30 AS RCEM_19, MODEL [C]LINE RATIOS.511 USING P21 AS RTAS_10, MODEL [C]LINE RATIOS.511 USING P22 AS RTAS_11, MODEL [C]LINE RATIOS.511 USING P23 AS RTAS_12, MODEL [C]LINE RATIOS.511 USING P24 AS RTAS_13, MODEL [C]LINE RATIOS.511 USING P25 AS RTAS_14, MODEL [C]LINE RATIOS.511 USING P26 AS RTAS_15, MODEL [C]LINE RATIOS.511 USING P27 AS RTAS_16, MODEL [C]LINE RATIOS.511 USING P28 AS RTAS_17, MODEL [C]LINE RATIOS.511 USING P29 AS RTAS_18, MODEL [C]LINE RATIOS.511 USING P30 AS RTAS_19, MODEL [C]LINE RATIOS.526 USING P21 AS ROE, MODEL [C]LINE RATIOS.526 USING P22 AS ROE_1, MODEL [C]LINE RATIOS.526 USING P23 AS ROE_2, MODEL [C]LINE RATIOS.526 USING P24 AS ROE_3, MODEL [C]LINE RATIOS.526 USING P25 AS ROE_4, MODEL [C]LINE RATIOS.526 USING P26 AS ROE_5, MODEL [C]LINE RATIOS.526 USING P27 AS ROE_6, MODEL [C]LINE RATIOS.526 USING P28 AS ROE_7, MODEL [C]LINE RATIOS.526 USING P29 AS ROE_8, MODEL [C]LINE RATIOS.526 USING P30 AS ROE_9, MODEL [C]LINE RATIOS.528 USING P21 AS ROCE, MODEL [C]LINE RATIOS.528 USING P22 AS ROCE_1, MODEL [C]LINE RATIOS.528 USING P23 AS ROCE_2, MODEL [C]LINE RATIOS.528 USING P24 AS ROCE_3, MODEL [C]LINE RATIOS.528 USING P25 AS ROCE_4, MODEL [C]LINE RATIOS.528 USING P26 AS ROCE_5, MODEL [C]LINE RATIOS.528 USING P27 AS ROCE_6, MODEL [C]LINE RATIOS.528 USING P28 AS ROCE_7, MODEL [C]LINE RATIOS.528 USING P29 AS ROCE_8, MODEL [C]LINE RATIOS.528 USING P30 AS ROCE_9, MODEL [C]LINE RATIOS.527 USING P21 AS ROA, MODEL [C]LINE RATIOS.527 USING P22 AS ROA_1, MODEL [C]LINE RATIOS.527 USING P23 AS ROA_2, MODEL [C]LINE RATIOS.527 USING P24 AS ROA_3, MODEL [C]LINE RATIOS.527 USING P25 AS ROA_4, MODEL [C]LINE RATIOS.527 USING P26 AS ROA_5, MODEL [C]LINE RATIOS.527 USING P27 AS ROA_6, MODEL [C]LINE RATIOS.527 USING P28 AS ROA_7, MODEL [C]LINE RATIOS.527 USING P29 AS ROA_8, MODEL [C]LINE RATIOS.527 USING P30 AS ROA_9, MODEL [C]LINE RATIOS.508 USING P21 AS PRMA_10, MODEL [C]LINE RATIOS.508 USING P22 AS PRMA_11, MODEL [C]LINE RATIOS.508 USING P23 AS PRMA_12, MODEL [C]LINE RATIOS.508 USING P24 AS PRMA_13, MODEL [C]LINE RATIOS.508 USING P25 AS PRMA_14, MODEL [C]LINE RATIOS.508 USING P26 AS PRMA_15, MODEL [C]LINE RATIOS.508 USING P27 AS PRMA_16, MODEL [C]LINE RATIOS.508 USING P28 AS PRMA_17, MODEL [C]LINE RATIOS.508 USING P29 AS PRMA_18, MODEL [C]LINE RATIOS.508 USING P30 AS PRMA_19, MODEL [C]LINE RATIOS.522 USING P21 AS GRMA, MODEL [C]LINE RATIOS.522 USING P22 AS GRMA_1, MODEL [C]LINE RATIOS.522 USING P23 AS GRMA_2, MODEL [C]LINE RATIOS.522 USING P24 AS GRMA_3, MODEL [C]LINE RATIOS.522 USING P25 AS GRMA_4, MODEL [C]LINE RATIOS.522 USING P26 AS GRMA_5, MODEL [C]LINE RATIOS.522 USING P27 AS GRMA_6, MODEL [C]LINE RATIOS.522 USING P28 AS GRMA_7, MODEL [C]LINE RATIOS.522 USING P29 AS GRMA_8, MODEL [C]LINE RATIOS.522 USING P30 AS GRMA_9, MODEL [C]LINE RATIOS.523 USING P21 AS ETMA, MODEL [C]LINE RATIOS.523 USING P22 AS ETMA_1, MODEL [C]LINE RATIOS.523 USING P23 AS ETMA_2, MODEL [C]LINE RATIOS.523 USING P24 AS ETMA_3, MODEL [C]LINE RATIOS.523 USING P25 AS ETMA_4, MODEL [C]LINE RATIOS.523 USING P26 AS ETMA_5, MODEL [C]LINE RATIOS.523 USING P27 AS ETMA_6, MODEL [C]LINE RATIOS.523 USING P28 AS ETMA_7, MODEL [C]LINE RATIOS.523 USING P29 AS ETMA_8, MODEL [C]LINE RATIOS.523 USING P30 AS ETMA_9, MODEL [C]LINE RATIOS.524 USING P21 AS EBMA, MODEL [C]LINE RATIOS.524 USING P22 AS EBMA_1, MODEL [C]LINE RATIOS.524 USING P23 AS EBMA_2, MODEL [C]LINE RATIOS.524 USING P24 AS EBMA_3, MODEL [C]LINE RATIOS.524 USING P25 AS EBMA_4, MODEL [C]LINE RATIOS.524 USING P26 AS EBMA_5, MODEL [C]LINE RATIOS.524 USING P27 AS EBMA_6, MODEL [C]LINE RATIOS.524 USING P28 AS EBMA_7, MODEL [C]LINE RATIOS.524 USING P29 AS EBMA_8, MODEL [C]LINE RATIOS.524 USING P30 AS EBMA_9, MODEL [C]LINE RATIOS.521 USING P21 AS CFOP, MODEL [C]LINE RATIOS.521 USING P22 AS CFOP_1, MODEL [C]LINE RATIOS.521 USING P23 AS CFOP_2, MODEL [C]LINE RATIOS.521 USING P24 AS CFOP_3, MODEL [C]LINE RATIOS.521 USING P25 AS CFOP_4, MODEL [C]LINE RATIOS.521 USING P26 AS CFOP_5, MODEL [C]LINE RATIOS.521 USING P27 AS CFOP_6, MODEL [C]LINE RATIOS.521 USING P28 AS CFOP_7, MODEL [C]LINE RATIOS.521 USING P29 AS CFOP_8, MODEL [C]LINE RATIOS.521 USING P30 AS CFOP_9, MODEL [C]LINE RATIOS.529 USING P21 AS EVET, MODEL [C]LINE RATIOS.529 USING P22 AS EVET_1, MODEL [C]LINE RATIOS.529 USING P23 AS EVET_2, MODEL [C]LINE RATIOS.529 USING P24 AS EVET_3, MODEL [C]LINE RATIOS.529 USING P25 AS EVET_4, MODEL [C]LINE RATIOS.529 USING P26 AS EVET_5, MODEL [C]LINE RATIOS.529 USING P27 AS EVET_6, MODEL [C]LINE RATIOS.529 USING P28 AS EVET_7, MODEL [C]LINE RATIOS.529 USING P29 AS EVET_8, MODEL [C]LINE RATIOS.529 USING P30 AS EVET_9, MODEL [C]LINE RATIOS.530 USING P21 AS MKCF, MODEL [C]LINE RATIOS.530 USING P22 AS MKCF_1, MODEL [C]LINE RATIOS.530 USING P23 AS MKCF_2, MODEL [C]LINE RATIOS.530 USING P24 AS MKCF_3, MODEL [C]LINE RATIOS.530 USING P25 AS MKCF_4, MODEL [C]LINE RATIOS.530 USING P26 AS MKCF_5, MODEL [C]LINE RATIOS.530 USING P27 AS MKCF_6, MODEL [C]LINE RATIOS.530 USING P28 AS MKCF_7, MODEL [C]LINE RATIOS.530 USING P29 AS MKCF_8, MODEL [C]LINE RATIOS.530 USING P30 AS MKCF_9, MODEL [C]LINE RATIOS.515 USING P21 AS NAT, MODEL [C]LINE RATIOS.515 USING P22 AS NAT_1, MODEL [C]LINE RATIOS.515 USING P23 AS NAT_2, MODEL [C]LINE RATIOS.515 USING P24 AS NAT_3, MODEL [C]LINE RATIOS.515 USING P25 AS NAT_4, MODEL [C]LINE RATIOS.515 USING P26 AS NAT_5, MODEL [C]LINE RATIOS.515 USING P27 AS NAT_6, MODEL [C]LINE RATIOS.515 USING P28 AS NAT_7, MODEL [C]LINE RATIOS.515 USING P29 AS NAT_8, MODEL [C]LINE RATIOS.515 USING P30 AS NAT_9, MODEL [C]LINE RATIOS.520 USING P21 AS IC, MODEL [C]LINE RATIOS.520 USING P22 AS IC_1, MODEL [C]LINE RATIOS.520 USING P23 AS IC_2, MODEL [C]LINE RATIOS.520 USING P24 AS IC_3, MODEL [C]LINE RATIOS.520 USING P25 AS IC_4, MODEL [C]LINE RATIOS.520 USING P26 AS IC_5, MODEL [C]LINE RATIOS.520 USING P27 AS IC_6, MODEL [C]LINE RATIOS.520 USING P28 AS IC_7, MODEL [C]LINE RATIOS.520 USING P29 AS IC_8, MODEL [C]LINE RATIOS.520 USING P30 AS IC_9, MODEL [C]LINE RATIOS.512 USING P21 AS STOT, MODEL [C]LINE RATIOS.512 USING P22 AS STOT_1, MODEL [C]LINE RATIOS.512 USING P23 AS STOT_2, MODEL [C]LINE RATIOS.512 USING P24 AS STOT_3, MODEL [C]LINE RATIOS.512 USING P25 AS STOT_4, MODEL [C]LINE RATIOS.512 USING P26 AS STOT_5, MODEL [C]LINE RATIOS.512 USING P27 AS STOT_6, MODEL [C]LINE RATIOS.512 USING P28 AS STOT_7, MODEL [C]LINE RATIOS.512 USING P29 AS STOT_8, MODEL [C]LINE RATIOS.512 USING P30 AS STOT_9, MODEL [C]LINE RATIOS.513 USING P21 AS COLL, MODEL [C]LINE RATIOS.513 USING P22 AS COLL_1, MODEL [C]LINE RATIOS.513 USING P23 AS COLL_2, MODEL [C]LINE RATIOS.513 USING P24 AS COLL_3, MODEL [C]LINE RATIOS.513 USING P25 AS COLL_4, MODEL [C]LINE RATIOS.513 USING P26 AS COLL_5, MODEL [C]LINE RATIOS.513 USING P27 AS COLL_6, MODEL [C]LINE RATIOS.513 USING P28 AS COLL_7, MODEL [C]LINE RATIOS.513 USING P29 AS COLL_8, MODEL [C]LINE RATIOS.513 USING P30 AS COLL_9, MODEL [C]LINE RATIOS.514 USING P21 AS CRPE, MODEL [C]LINE RATIOS.514 USING P22 AS CRPE_1, MODEL [C]LINE RATIOS.514 USING P23 AS CRPE_2, MODEL [C]LINE RATIOS.514 USING P24 AS CRPE_3, MODEL [C]LINE RATIOS.514 USING P25 AS CRPE_4, MODEL [C]LINE RATIOS.514 USING P26 AS CRPE_5, MODEL [C]LINE RATIOS.514 USING P27 AS CRPE_6, MODEL [C]LINE RATIOS.514 USING P28 AS CRPE_7, MODEL [C]LINE RATIOS.514 USING P29 AS CRPE_8, MODEL [C]LINE RATIOS.514 USING P30 AS CRPE_9, MODEL [C]LINE RATIOS.525 USING P21 AS EXOP, MODEL [C]LINE RATIOS.525 USING P22 AS EXOP_1, MODEL [C]LINE RATIOS.525 USING P23 AS EXOP_2, MODEL [C]LINE RATIOS.525 USING P24 AS EXOP_3, MODEL [C]LINE RATIOS.525 USING P25 AS EXOP_4, MODEL [C]LINE RATIOS.525 USING P26 AS EXOP_5, MODEL [C]LINE RATIOS.525 USING P27 AS EXOP_6, MODEL [C]LINE RATIOS.525 USING P28 AS EXOP_7, MODEL [C]LINE RATIOS.525 USING P29 AS EXOP_8, MODEL [C]LINE RATIOS.525 USING P30 AS EXOP_9, MODEL [C]LINE RATIOS.531 USING P21 AS RDOP, MODEL [C]LINE RATIOS.531 USING P22 AS RDOP_1, MODEL [C]LINE RATIOS.531 USING P23 AS RDOP_2, MODEL [C]LINE RATIOS.531 USING P24 AS RDOP_3, MODEL [C]LINE RATIOS.531 USING P25 AS RDOP_4, MODEL [C]LINE RATIOS.531 USING P26 AS RDOP_5, MODEL [C]LINE RATIOS.531 USING P27 AS RDOP_6, MODEL [C]LINE RATIOS.531 USING P28 AS RDOP_7, MODEL [C]LINE RATIOS.531 USING P29 AS RDOP_8, MODEL [C]LINE RATIOS.531 USING P30 AS RDOP_9, MODEL [C]LINE RATIOS.500 USING P21 AS CURR_10, MODEL [C]LINE RATIOS.500 USING P22 AS CURR_11, MODEL [C]LINE RATIOS.500 USING P23 AS CURR_12, MODEL [C]LINE RATIOS.500 USING P24 AS CURR_13, MODEL [C]LINE RATIOS.500 USING P25 AS CURR_14, MODEL [C]LINE RATIOS.500 USING P26 AS CURR_15, MODEL [C]LINE RATIOS.500 USING P27 AS CURR_16, MODEL [C]LINE RATIOS.500 USING P28 AS CURR_17, MODEL [C]LINE RATIOS.500 USING P29 AS CURR_18, MODEL [C]LINE RATIOS.500 USING P30 AS CURR_19, MODEL [C]LINE RATIOS.501 USING P21 AS LIQR, MODEL [C]LINE RATIOS.501 USING P22 AS LIQR_1, MODEL [C]LINE RATIOS.501 USING P23 AS LIQR_2, MODEL [C]LINE RATIOS.501 USING P24 AS LIQR_3, MODEL [C]LINE RATIOS.501 USING P25 AS LIQR_4, MODEL [C]LINE RATIOS.501 USING P26 AS LIQR_5, MODEL [C]LINE RATIOS.501 USING P27 AS LIQR_6, MODEL [C]LINE RATIOS.501 USING P28 AS LIQR_7, MODEL [C]LINE RATIOS.501 USING P29 AS LIQR_8, MODEL [C]LINE RATIOS.501 USING P30 AS LIQR_9, MODEL [C]LINE RATIOS.502 USING P21 AS SHLQ, MODEL [C]LINE RATIOS.502 USING P22 AS SHLQ_1, MODEL [C]LINE RATIOS.502 USING P23 AS SHLQ_2, MODEL [C]LINE RATIOS.502 USING P24 AS SHLQ_3, MODEL [C]LINE RATIOS.502 USING P25 AS SHLQ_4, MODEL [C]LINE RATIOS.502 USING P26 AS SHLQ_5, MODEL [C]LINE RATIOS.502 USING P27 AS SHLQ_6, MODEL [C]LINE RATIOS.502 USING P28 AS SHLQ_7, MODEL [C]LINE RATIOS.502 USING P29 AS SHLQ_8, MODEL [C]LINE RATIOS.502 USING P30 AS SHLQ_9, MODEL [C]LINE RATIOS.503 USING P21 AS SOLR_10, MODEL [C]LINE RATIOS.503 USING P22 AS SOLR_11, MODEL [C]LINE RATIOS.503 USING P23 AS SOLR_12, MODEL [C]LINE RATIOS.503 USING P24 AS SOLR_13, MODEL [C]LINE RATIOS.503 USING P25 AS SOLR_14, MODEL [C]LINE RATIOS.503 USING P26 AS SOLR_15, MODEL [C]LINE RATIOS.503 USING P27 AS SOLR_16, MODEL [C]LINE RATIOS.503 USING P28 AS SOLR_17, MODEL [C]LINE RATIOS.503 USING P29 AS SOLR_18, MODEL [C]LINE RATIOS.503 USING P30 AS SOLR_19, MODEL [C]LINE RATIOS.SOLL USING P21 AS SOLL, MODEL [C]LINE RATIOS.SOLL USING P22 AS SOLL_1, MODEL [C]LINE RATIOS.SOLL USING P23 AS SOLL_2, MODEL [C]LINE RATIOS.SOLL USING P24 AS SOLL_3, MODEL [C]LINE RATIOS.SOLL USING P25 AS SOLL_4, MODEL [C]LINE RATIOS.SOLL USING P26 AS SOLL_5, MODEL [C]LINE RATIOS.SOLL USING P27 AS SOLL_6, MODEL [C]LINE RATIOS.SOLL USING P28 AS SOLL_7, MODEL [C]LINE RATIOS.SOLL USING P29 AS SOLL_8, MODEL [C]LINE RATIOS.SOLL USING P30 AS SOLL_9, MODEL [C]LINE RATIOS.504 USING P21 AS GEAR, MODEL [C]LINE RATIOS.504 USING P22 AS GEAR_1, MODEL [C]LINE RATIOS.504 USING P23 AS GEAR_2, MODEL [C]LINE RATIOS.504 USING P24 AS GEAR_3, MODEL [C]LINE RATIOS.504 USING P25 AS GEAR_4, MODEL [C]LINE RATIOS.504 USING P26 AS GEAR_5, MODEL [C]LINE RATIOS.504 USING P27 AS GEAR_6, MODEL [C]LINE RATIOS.504 USING P28 AS GEAR_7, MODEL [C]LINE RATIOS.504 USING P29 AS GEAR_8, MODEL [C]LINE RATIOS.504 USING P30 AS GEAR_9, MODEL [C]LINE RATIOS.519 USING P2 AS PPE, MODEL [C]LINE RATIOS.519 USING P4 AS PPE_1, MODEL [C]LINE RATIOS.519 USING P6 AS PPE_2, MODEL [C]LINE RATIOS.519 USING P8 AS PPE_3, MODEL [C]LINE RATIOS.519 USING P10 AS PPE_4, MODEL [C]LINE RATIOS.519 USING P12 AS PPE_5, MODEL [C]LINE RATIOS.519 USING P14 AS PPE_6, MODEL [C]LINE RATIOS.519 USING P16 AS PPE_7, MODEL [C]LINE RATIOS.519 USING P18 AS PPE_8, MODEL [C]LINE RATIOS.519 USING P20 AS PPE_9, MODEL [C]LINE RATIOS.517 USING P2 AS TPE, MODEL [C]LINE RATIOS.516 USING P21 AS SCT, MODEL [C]LINE RATIOS.516 USING P22 AS SCT_1, MODEL [C]LINE RATIOS.516 USING P23 AS SCT_2, MODEL [C]LINE RATIOS.516 USING P24 AS SCT_3, MODEL [C]LINE RATIOS.516 USING P25 AS SCT_4, MODEL [C]LINE RATIOS.516 USING P26 AS SCT_5, MODEL [C]LINE RATIOS.516 USING P27 AS SCT_6, MODEL [C]LINE RATIOS.516 USING P28 AS SCT_7, MODEL [C]LINE RATIOS.516 USING P29 AS SCT_8, MODEL [C]LINE RATIOS.516 USING P30 AS SCT_9, MODEL [C]LINE RATIOS.518 USING P2 AS ACE, MODEL [C]LINE RATIOS.505 USING P2 AS SFPE, MODEL [C]LINE RATIOS.506 USING P2 AS WCPE, MODEL [C]LINE RATIOS.507 USING P2 AS TAPE FROM RemoteAccess.A"""

In [13]:
get_custom_data(q, "DEFEB13190", "financials", "Commerzbank")

1DN1EE24B4MM5W0
1DN1EE24B4MM5W0
0
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><soap:Fault><soap:Code><soap:Value>soap:Receiver</soap:Value></soap:Code><soap:Reason><soap:Text xml:lang="en">System.Web.Services.Protocols.SoapException: Server was unable to process request. ---&gt; Credits limitations
   at System.Web.Services.Protocols.SoapHttpClientProtocol.ReadResponse(SoapClientMessage message, WebResponse response, Stream responseStream, Boolean asyncCall)
   at System.Web.Services.Protocols.SoapHttpClientProtocol.Invoke(String methodName, Object[] parameters)
   at ProxyRemoteAccess.com.bvdep.webservices.RemoteAccess.GetData(String sessionHandle, SelectionResult selection, String query, Int32 fromRecord, Int32 nrRecords, ResultFormatting resultFormat)
   at ProxyRemoteAccess.RemoteAccess.GetData(String sessionHand